In [1]:
# установка внешних программ и библиотек на виртуальную машину
!git clone --recurse-submodules https://github.com/KarlsruheMIS/KaMIS.git
%cd KaMIS
!./compile_withcmake.sh

%cd ..


!pip3 install python-sat
#!sudo apt install minisat
!git clone https://github.com/arminbiere/kissat.git
%cd kissat
!./configure
!make

%cd ..

!cp KaMIS/deploy/redumis redumis
!cp KaMIS/deploy/online_mis online_mis
!cp kissat/build/kissat kissat1



Cloning into 'KaMIS'...
remote: Enumerating objects: 1481, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 1481 (delta 39), reused 45 (delta 27), pack-reused 1399
Receiving objects: 100% (1481/1481), 12.78 MiB | 12.74 MiB/s, done.
Resolving deltas: 100% (830/830), done.
/content/KaMIS
-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Performing Test COMPILER_SUPPORTS_FUNROLL_LOOPS
-- Performing Test COMPILER_SUPPORTS_FUNROLL_LOOPS - Success
-- Performing Test COMPILER_SUPP

In [1]:
# функции из библиотеки
# раскраска графа в формате  DIMACS, преобразование в CNF

import math
import numpy as np
from numpy import linalg as LA
import networkx as nx

#import matplotlib.pylab as plt

#from google.colab import files 
#files.upload() 
#files.download("file.txt")

#https://pysathq.github.io/
#!pip install python-sat
#import pysat
from pysat.formula import CNF
from pysat.solvers import *

import os
import sys
import random
from zipfile import ZipFile
#from tqdm import tqdm_notebook as tqdm
from itertools import combinations, permutations
import glob

out_dir = ""

class Utils:
    
    def read_dimacs_graph(file = 'graph.col'):
        
        if not (os.path.exists(file) and os.path.getsize(file) > 0):        
            raise Exception("File " + file + " not found")
        
        nodes = []    
        edges = []
        labels = []
        got_labels = False
        nnodes = nedges = 0
        
        with open(file, 'r') as f:
            for line in f:
                line = [l.strip() for l in line.split(' ')]
                if line[0] == 'c': #comment
                    continue
                elif line[0] == 'p': #problem
                    nnodes = int(line[2])
                    nedges = int(line[3])
                    nodes = list(range(1, nnodes + 1))
                    labels = [0] * nnodes
                elif line[0] == 'e': #edge
                    edges.append((int(line[1]), int(line[2])))
                elif line[0] == 'l':
                    labels[int(line[1]) - 1] = int(line[2])
                    got_labels = True

        if got_labels:        
            nodes = [(n, {'c' : l}) for n, l in zip(nodes, labels)]

        g = nx.Graph()
        g.add_nodes_from(nodes)
        g.add_edges_from(edges)
        return g

    def write_dimacs_graph(file = 'graph.col', g = nx.Graph(), comments = []):
        with open(file, 'w') as f:
            for c in comments:
                f.write("c " + c + "\n")
            f.write("p EDGE {} {}\n".format(g.number_of_nodes(), g.number_of_edges()))
            for u, v in g.edges():
                f.write("e {} {}\n".format(u, v))
            for node in g.nodes():
                if 'c' in g.node[node]:
                    f.write("l {} {}\n".format(node, g.node[node]['c']))

    def draw_with_colors(g = nx.Graph()):
        color_map = []
        for node in g.nodes():
            if 'c' in g.node[node]:
                color_map.append(g.node[node]['c'] * 10)            
        nx.draw(g, pos = nx.spring_layout(g, scale=2), node_color=color_map, with_labels=True, cmap = plt.cm.jet)

    def write_proof(file = "proof.txt", proof = []):
        with open(file, 'w') as f:
            for p in proof:
                f.write("%s\n" % str(p))

    def zip_files(file = "archive.zip", files = []):
        with ZipFile(file, 'w') as archive:
            for f in set(files):                
                if not (os.path.exists(file) and os.path.getsize(file) > 0):        
                    raise Exception("File " + file + " not found")
                archive.write(f)
                
def find_triangle(g = nx.Graph()):
    for a in g:
        for b, c in combinations(g[a], 2):
            if b in g[c]:
                return [a, b, c]
    return []
             
    #return set(frozenset([a, b, c]) for a in g for b, c in combinations(g[a], 2) if b in g[c])


def find_isolates(g = nx.Graph()):
    isolates = []
    for n in g:
        if g.degree(n) == 0:
            isolates.append(n)
    return isolates

class ColMap:
    
    def __init__(self, g = nx.Graph(), ncolors = 40):
        
        self.ncolors = ncolors
        self.cmap = dict()
        self.cunmap = dict()
    
        i = 1
        for node in g.nodes():
            for color in range(1, ncolors + 1):            
                self.cmap[(node, color)] = i
                self.cunmap[i] = (node, color)
                i += 1    

    def enc(self, node, color):
        return self.cmap[(node, color)]

    def dec(self, node_color):
        return self.cunmap[node_color]

class ColSAT:
   
    def __init__(self, g = nx.Graph(), ncolors = 10):
        
        self.ncolors = ncolors
        self.g = g.copy()
        self.cmap = ColMap(g, ncolors)        

    def check_coloring(self):
        for n1, n2 in self.g.edges():
            if 'c' not in self.g.node[n1] or 'c' not in self.g.node[n2]:
                return False
            if self.g.node[n1]['c'] == self.g.node[n2]['c']:
                return False
        return True
    
    def apply_model(self):
        
        check = set()
        for var in self.model[self.model > 0]:        
            node, color = self.cmap.dec(var)
            self.g.node[node]['c'] = color
            if (node, color) in check:
                raise Exception("Two colors for one node???")
            else:
                check.add((node, color))
        
        self.colored = self.check_coloring()
        
        if self.colored != self.solved:
            raise Exception("Something went wrong!")
        
        return self.colored
        
    def build_cnf(self, ):
        
        self.formula = CNF()
        colors = list(range(1, self.ncolors + 1))    

        maxc = -1        # condition k<12 is not for general case
        k = 0
        for clique in nx.find_cliques(self.g):
            if len(clique)>maxc:
                maxclique = clique.copy()  
                maxc = len(clique)
            if k<12:
                k += 1
                continue
           
            col = 1
            if len(maxclique)>self.ncolors:
                print("Size of the maximal clique is {}".format(len(maxclique)))
                raise Exception("Size of the maximal clique is {} > {}".format(len(maxclique),ncolors))
            for v in maxclique:
                self.formula.append([self.cmap.enc(v, col)])
                col += 1
            break

        for n1, n2 in self.g.edges():
            for c in colors:            
                self.formula.append([-self.cmap.enc(n1, c), -self.cmap.enc(n2, c)])


        for n in self.g.nodes():
            #if not n in specials:
            self.formula.append([self.cmap.enc(n, c) for c in colors])
            #for c1 in colors:
            #    for c2 in colors:
            #        if c1 < c2:
            #            self.formula.append([-self.cmap.enc(n, c1), -self.cmap.enc(n, c2)])
        
        return self.formula
    
    def solve_cnf(self, solver = ''):
        
        triangle = find_triangle(self.g)
        assumptions = []
        if len(triangle) > 0:            
            assumptions = [self.cmap.enc(triangle[0], 1), self.cmap.enc(triangle[1], 2), self.cmap.enc(triangle[2], 3)]
            
        #Glucose3, Glucose4, Lingeling, MapleChrono, MapleCM, Maplesat, Minisat22, MinisatGH
        #with Glucose4(bootstrap_with=self.formula.clauses, with_proof=True) as ms:        
        with Lingeling(bootstrap_with=self.formula.clauses) as ms:
            self.solved = ms.solve(assumptions=assumptions)
            if self.solved:
                self.model = np.array(ms.get_model())
                self.apply_model()
            else:                
                self.proof = []#ms.get_proof()
                self.colored = False
                
        return self.solved






In [2]:
# выбор графа на вершинах гиперкуба (покрышки), удаление независимого множества,
# проверка существования раскраски

import networkx as nx

def dist(a,b):
  return sum([int(c) for c in bin(a^b).split('b')[1]])

def bin_graph(n, k, trim = []):   # третий параметр - множество вершин
  g = nx.Graph()                  # по k-окрестностям которых происходит отсечение    
  for i in range(0,2**n):
    if dist(0,i)%2==1:
      continue
    flag = True
    for t in trim:
      if dist(t,i)>k:
        flag = False
        break
    if flag:
      g.add_node(i)
  for i in list(g.nodes()):
    for j in g.nodes():
      if (i<j) and (dist(i,j)==k):
        g.add_edge(i,j)
  return nx.convert_node_labels_to_integers(g)

def small_graph(n, k, dists = [2,4,8]):   # третий параметр - множество вершин
  g = nx.Graph()                  # по k-окрестностям которых происходит отсечение    
  for i in range(0,2**n):
    if dist(0,i) not in dists:
      continue
    g.add_node(i)
  for i in list(g.nodes()):
    for j in g.nodes():
      if (i<j) and (dist(i,j)==k):
        g.add_edge(i,j)
  return nx.convert_node_labels_to_integers(g)

def bin_graph1(n, k, trim = []):
  g = nx.Graph()
  for i in range(0,2**n):
    if dist(0,i)%2==1:
      continue
    flag = True
    if dist(0,i)>k:
      flag = False
      
    if flag or i in set(trim):
      g.add_node(i)
  for i in list(g.nodes()):
    for j in g.nodes():
      if (i<j) and (dist(i,j)==k):
        g.add_edge(i,j)
  return nx.convert_node_labels_to_integers(g)


def r_graph(n, k):      # граф запретов,
  g = nx.Graph()        # ребрами соединены вершины на расстоянии >k
  for i in range(0,2**n):
    g.add_node(i)
  for i in list(g.nodes()):
    for j in g.nodes():
      if (i<j) and (dist(i,j)>k):
        g.add_edge(i,j)
  return g

def write_metis(g0,fname):    # запись графа во входном формате KaMIS
  g = nx.convert_node_labels_to_integers(g0)
  with open(fname,"w") as f:
    f.write("{} {}\n".format(g.number_of_nodes(),g.number_of_edges()))
    for v in range(g.number_of_nodes()):
      f.write(" ".join([str(i+1) for i in sorted(list(g[v]))])+"\n")    

def write_cnf(g,ncolors,filename):  # запись КНФ для SAT-решателя    
  problem = ColSAT(g, ncolors)
  problem.build_cnf().to_file(filename)

def select_mis(g,filename):  # возвращает независимое множество
  g1 = g.copy()
  with open(filename,"r") as f:
    l = f.readlines()
    for i in range(len(l)):
      if int(l[i])==0:
        g1.remove_node(i)    
  return nx.convert_node_labels_to_integers(g1,0)      


def delete_mis(g,filename):  # удаляет независимосе множество
  g1 = g.copy()
  with open(filename,"r") as f:
    l = f.readlines()
    for i in range(len(l)):
      if int(l[i])==1:
        g1.remove_node(i)    
  return nx.convert_node_labels_to_integers(g1,0)      

# размерность, расстояние, номера вершин, по окрестностям которых 
# выполняется отсечение
 
r = bin_graph(10,6,trim=[0, 984, 730, 826, 1000, 718, 857])


#g = bin_graph(9,6,trim=[0,63])  
#print(r.number_of_nodes())
#print(r.number_of_edges())
#write_metis(r,"test.graph")

#!./redumis test.graph --output test.mis --time_limit=30

#g = r.copy()
g1 = r.copy()

print(g1.number_of_nodes())
print(g1.number_of_edges())

#write_cnf(g1,10,"test.cnf")

#!./kissat1 test.cnf

nmis = 0

283
15247


In [3]:
FILENAME = "test_11"
COLORSS = 0

def is_not_colorable(graph, color_count, timer):
    global FILENAME, COLORSS
    COLORSS += 1
    coloring_filename = FILENAME + ".cnf"
    try:
        graph_copy = graph.copy()
        write_cnf(graph_copy, color_count, coloring_filename)
    except Exception as err:
        return True

    outputt = !timeout $timer ./kissat1 $coloring_filename
    if any(["UNSATISFIABLE" in line for line in outputt]):
         return True
    return False

def is_colorable(graph, color_count, timer):
    global FILENAME, COLORSS
    COLORSS += 1
    coloring_filename = FILENAME + ".cnf"
    try:
        graph_copy = graph.copy()
        write_cnf(graph_copy, color_count, coloring_filename)
    except Exception as err:
        return False

    outputt = !timeout $timer ./kissat1 $coloring_filename
    if all(["UNSATISFIABLE" not in line for line in outputt]):
        if any(["SATISFIABLE" in line for line in outputt]):
            return True
    return False

def check_colorable_fast(graph, max_colors=100, timer=120):
    if graph.number_of_edges() == 0:
        return (None, 0)

    max_uncolorable = None
    min_colorable = None
    
    left_border = 1
    right_border = max_colors + 1
    while right_border - left_border > 1:
        color_count_to_check = (right_border + left_border) // 2
        if is_not_colorable(graph, color_count_to_check, timer):
            left_border = color_count_to_check
        else:
            right_border = color_count_to_check
    max_uncolorable = left_border

    left_border = 1
    right_border = max_colors
    while right_border - left_border > 1:
        color_count_to_check = (right_border + left_border) // 2
        if is_colorable(graph, color_count_to_check, timer):
            right_border = color_count_to_check
        else:
            left_border = color_count_to_check
    if right_border == max_colors:
        if is_colorable(graph, max_colors, timer):
            min_colorable = right_border 
        return (max_uncolorable, min_colorable)
    else:
        min_colorable = right_border
        return (max_uncolorable, min_colorable)
    return (None, None)

# my_graph = small_graph(10, 6)
# qq = my_graph.copy()
# check_colorable_fast(qq)

# Ниже рабочая зона (Да)#


In [4]:
FILENAME = "NoK4"

In [5]:
# !!! this is late-stage thing
base_conf = [0, 945, 910]

k2s = [(0, 945),
       (0, 910),
       (945, 910)]
kl3s = [(0, 910, 945)]

k_2 = []
k_4 = []
k_6 = []

for i in range(2**10):
    if dist(0, i) % 2 == 1:
        continue
    flag = True
    for base_v in base_conf:
        if dist(i, base_v) == 0 or dist(i, base_v) > 6:
            flag = False
            continue
    big_flag = True
    for kl3 in kl3s:
        fflag = False
        for v in kl3:
            if dist(i, v) < 6:
                fflag = True
                break
        if not fflag:
            big_flag = False
            break
    if not big_flag:
        continue
    if flag:
        if dist(0, i) == 2:
            k_2.append(i)
        if dist(0, i) == 4:
            k_4.append(i)
        if dist(0, i) == 6:
            k_6.append(i)

print(len(k_6), len(k_4), len(k_2))

150 160 33


In [46]:
# BIG_NUM = 3
# COUNTER = 0
# SAVED = []
# BPM = [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]

def simple_cook_graph_no_K4(base_conf, add_conf, kl3s, k_2, k_4, k_6, n=10, k=6):   # третий параметр - множество вершин
    global COUNTER, BPM
    COUNTER += 1
    if COUNTER % 50 == 0:
        print(BPM)
    g = nx.Graph() 
    bans = base_conf + add_conf
    pooles = k_2 + k_4 + k_6
    for i in bans:
        g.add_node(i)
    
    for i in pooles:
        flag = True
        for j in bans:
            if dist(i,j) > k or dist(i, j) == 0:
                flag = False
                break
        if not flag:
            continue

        big_flag = True
        for kl3 in kl3s:
            fflag = False
            for v in kl3:
                if dist(i, v) < 6:
                    fflag = True
                    break
            if not fflag:
                big_flag = False
                break
        if big_flag:
            g.add_node(i)
    for i in list(g.nodes()):
        for j in g.nodes():
            if (i<j) and (dist(i,j)==k):
                g.add_edge(i,j)
    return nx.convert_node_labels_to_integers(g)

def sort_to_check(to_check, base_conf, add_conf):
    to_to = [[a, 0] for a in to_check]
    for a in to_to:
        for b in base_conf:
            if dist(a[0], b) == 6:
                a[1] += 1
        for b in  add_conf:
            if dist(a[0], b) == 6:
                a[1] += 1
    tbd = sorted(to_to, key=lambda x: (-x[1], x[0]))
    return [a[0] for a in tbd]


def bruteforce_nolimits_nok4(to_check, base_conf, add_conf, k_2, k_4, k2s = [], kl3s=[], how_much=BIG_NUM, timer=120, n=10, k=6, fromm=0):
    global COUNTER, BPM, SAVED
    to_check = sort_to_check(to_check, base_conf, add_conf)
    if how_much == BIG_NUM:
        to_check = to_check[fromm:]
    ln = len(to_check)
    for i in range(ln):
        BPM[how_much][0] = i
        BPM[how_much][1] = ln
        # if i % 20 == 0 and how_much == BIG_NUM - 1:
        #     print(i, len(to_check))
        elem = to_check[i]
        if elem in base_conf or elem in add_conf:
            continue
        flag = True
        for el in add_conf:
            if dist(elem, el) > 6:
                flag = False
                break
        if not flag:
            continue
        big_flag = True
        for kl3 in kl3s:
            fflag = False
            for v in kl3:
                if dist(elem, v) < 6:
                    fflag = True
                    break
            if not fflag:
                big_flag = False
                break
        if not big_flag:
            continue
    
        kl3s_updated = []
        for k2ss in k2s:
            if dist(elem, k2ss[0]) == 6 and dist(elem, k2ss[1]) == 6:
                kl3s_updated.append((k2ss[0], k2ss[1], elem))
        k2s_updated = []
        for v in base_conf + add_conf:
            if dist(elem, v) == 6:
                k2s_updated.append((elem, v))
        if how_much > 1:
            bruteforce_nolimits_nok4(to_check[i+1:], base_conf, add_conf + [elem], k_2=k_2,
                                     k_4=k_4, k2s = k2s + k2s_updated, kl3s = kl3s + kl3s_updated,
                                     how_much=how_much-1,timer=timer, n=n, k=k)
        else:
            grph = simple_cook_graph_no_K4(base_conf=base_conf, add_conf=add_conf + [elem], kl3s = kl3s + kl3s_updated, k_2=k_2, k_4=k_4, k_6=to_check[i+1:], n=n, k=k)
            brb = is_colorable(grph, 11, timer)
            if not brb:
                SAVED.append((base_conf, add_conf + [elem]))
                print(base_conf, add_conf + [elem])
#     if how_much == BIG_NUM:
#         print("analysing {} finished".format(BIG_NUM))

def bruteforce_nolimits_nok4_short(to_check, base_conf, add_conf, k_2, k_4, k2s = [], kl3s=[], how_much=4, timer=120, n=10, k=6, fromm=0):
    global COUNTER, BPM, SAVED
    to_check = sort_to_check(to_check, base_conf, add_conf)
    if how_much == BIG_NUM:
        to_check = to_check[fromm:]
    ln = len(to_check)
    for i in range(ln):
        BPM[how_much][0] = i
        BPM[how_much][1] = ln
        elem = to_check[i]
        if elem in base_conf or elem in add_conf:
            continue
        flag = True
        for el in add_conf:
            if dist(elem, el) > 6:
                flag = False
                break
        if not flag:
            continue
        big_flag = True
        for kl3 in kl3s:
            fflag = False
            for v in kl3:
                if dist(elem, v) < 6:
                    fflag = True
                    break
            if not fflag:
                big_flag = False
                break
        if not big_flag:
            continue
    
        kl3s_updated = []
        for k2ss in k2s:
            if dist(elem, k2ss[0]) == 6 and dist(elem, k2ss[1]) == 6:
                kl3s_updated.append((k2ss[0], k2ss[1], elem))
        k2s_updated = []
        for v in base_conf + add_conf:
            if dist(elem, v) == 6:
                k2s_updated.append((elem, v))

        if how_much == 1:
            grph = simple_cook_graph_no_K4(base_conf=base_conf, add_conf=add_conf + [elem], kl3s = kl3s + kl3s_updated, k_2=k_2, k_4=k_4, k_6=[], n=n, k=k)
            brb = is_colorable(grph, 11, timer)
            if not brb:
                print(base_conf, add_conf + [elem])
                SAVED.append((base_conf, add_conf + [elem]))
        if how_much > 1:
            bruteforce_nolimits_nok4_short(to_check[i+1:], base_conf, add_conf + [elem], k_2=k_2,
                                     k_4=k_4, k2s = k2s + k2s_updated, kl3s = kl3s + kl3s_updated,
                                     how_much=how_much-1,timer=timer, n=n, k=k)

    if how_much == BIG_NUM:
        pass
#         print("analysing {}-or-less-but-at-least-1 finished".format(BIG_NUM))

In [7]:
kv = base_conf

In [ ]:
# %%time
# BIG_NUM=5
# print("COLORED", COLORSS)
# print('to check:', len(k_6), len(k_6))
# bruteforce_nolimits_nok4(k_6, kv, [], k_2, k_4, k2s, kl3s, BIG_NUM, 20, 10, 6)
# print("AFTER ALL-COLORED", COLORSS)

In [ ]:
# %%time
# BIG_NUM=4
# print("COLORED", COLORSS)
# print('to check:', len(k_6), len(k_6))
# bruteforce_nolimits_nok4_short(k_6, kv, [], k_2, k_4, k2s, kl3s, BIG_NUM, 20, 10, 6)
# print("AFTER ALL-COLORED", COLORSS)

In [10]:
%%time
BIG_NUM=2
print("COLORED", COLORSS)
print('to check:', len(k_6), len(k_6))
bruteforce_nolimits_nok4_short(k_6, kv, [], k_2, k_4, k2s, kl3s, BIG_NUM, 20, 10, 6)
print("AFTER ALL-COLORED", COLORSS)

COLORED 150
to check: 150 150
[[0, 0], [65, 149], [0, 150], [0, 0], [0, 0], [0, 0]]
[[0, 0], [126, 149], [0, 150], [0, 0], [0, 0], [0, 0]]
[[0, 0], [46, 148], [1, 150], [0, 0], [0, 0], [0, 0]]
[[0, 0], [105, 148], [1, 150], [0, 0], [0, 0], [0, 0]]
[[0, 0], [18, 147], [2, 150], [0, 0], [0, 0], [0, 0]]
[[0, 0], [78, 147], [2, 150], [0, 0], [0, 0], [0, 0]]
[[0, 0], [139, 147], [2, 150], [0, 0], [0, 0], [0, 0]]
[[0, 0], [61, 146], [3, 150], [0, 0], [0, 0], [0, 0]]
[[0, 0], [122, 146], [3, 150], [0, 0], [0, 0], [0, 0]]
[[0, 0], [45, 145], [4, 150], [0, 0], [0, 0], [0, 0]]
[[0, 0], [104, 145], [4, 150], [0, 0], [0, 0], [0, 0]]
[[0, 0], [21, 144], [5, 150], [0, 0], [0, 0], [0, 0]]
[[0, 0], [80, 144], [5, 150], [0, 0], [0, 0], [0, 0]]
[[0, 0], [0, 143], [6, 150], [0, 0], [0, 0], [0, 0]]
[[0, 0], [66, 143], [6, 150], [0, 0], [0, 0], [0, 0]]
[[0, 0], [127, 143], [6, 150], [0, 0], [0, 0], [0, 0]]
[[0, 0], [53, 142], [7, 150], [0, 0], [0, 0], [0, 0]]
[[0, 0], [114, 142], [7, 150], [0, 0], [0, 0], 

In [9]:
%%time
BIG_NUM=1
print("COLORED", COLORSS)
print('to check:', len(k_6), len(k_6))
bruteforce_nolimits_nok4_short(k_6, kv, [], k_2, k_4, k2s, kl3s, BIG_NUM, 20, 10, 6)
print("AFTER ALL-COLORED", COLORSS)


COLORED 0
to check: 150 150
[[0, 0], [49, 150], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [99, 150], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [149, 150], [0, 0], [0, 0], [0, 0], [0, 0]]
analysing 1-or-less-but-at-least-1 finished
AFTER ALL-COLORED 150
CPU times: user 1min 2s, sys: 3.1 s, total: 1min 5s
Wall time: 1min 30s


In [47]:
# на самом деле не симпл
def simple_cook_graph_no_K4(base_conf, add_conf, kl3s, k_2, k_4, k_6, n=10, k=6):   # третий параметр - множество вершин
    global COUNTER, BPM
    COUNTER += 1
    if COUNTER % 50 == 0:
        print(BPM)
    g = nx.Graph() 
    bans = base_conf + add_conf
    pooles = k_2 + k_4 + k_6
    for i in bans:
        g.add_node(i)
    
    good_vrtx = []
    for i in pooles:
        flag = True
        for j in bans:
            if dist(i,j) > k or dist(i, j) == 0:
                flag = False
                break
        if not flag:
            continue

        big_flag = True
        for kl3 in kl3s:
            fflag = False
            for v in kl3:
                if dist(i, v) < 6:
                    fflag = True
                    break
            if not fflag:
                big_flag = False
                break
        if big_flag:
            g.add_node(i)
            good_vrtx.append(i)

    prepared = []
    for i in bans:
        for j in bans:
            if (i<j) and (dist(i,j)==k):
                g.add_edge(i,j)
                prepared.append((i, j))
    
    for i in bans:
        for j in good_vrtx:
            if (i<j) and (dist(i,j)==k):
                g.add_edge(i,j)

    for i in good_vrtx:
        for j in bans:
            if (i<j) and (dist(i,j)==k):
                g.add_edge(i,j)

    for i in good_vrtx:
        for j in good_vrtx:
            if (i<j) and (dist(i,j)==k):
                flag = True
                for prep in prepared:
                    if dist(i, prep[0]) == k and dist(i, prep[1]) == k and dist(j, prep[0]) == k and dist(j, prep[1]) == k:
                        flag = False
                if flag:
                    g.add_edge(i,j)
    

    # for i in list(g.nodes()):
    #     for j in g.nodes():
    #         if (i<j) and (dist(i,j)==k):
    #             g.add_edge(i,j)
    return nx.convert_node_labels_to_integers(g)

In [11]:
%%time
BIG_NUM=3
print("COLORED", COLORSS)
print('to check:', len(k_6), len(k_6))
bruteforce_nolimits_nok4(k_6[:80], kv, [], k_2+k_6[80:], k_4, k2s, kl3s, BIG_NUM, 20, 10, 6)
print("AFTER ALL-COLORED", COLORSS)

COLORED 17
to check: 150 150
[[0, 0], [45, 78], [0, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], [37, 77], [1, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], [29, 75], [3, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], [19, 72], [6, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], [15, 71], [7, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], [13, 70], [8, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], [11, 69], [9, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], [6, 68], [10, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], [5, 66], [12, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], [3, 64], [14, 79], [0, 80], [0, 0], [0, 0]]
[0, 945, 910] [159, 175, 189]
[[0, 0], [58, 64], [14, 79], [0, 80], [0, 0], [0, 0]]
[0, 945, 910] [159, 183, 189]
[0, 945, 910] [159, 183, 190]
[[0, 0], [46, 63], [15, 79], [0, 80], [0, 0], [0, 0]]
^C
[0, 945, 910] [159, 187, 717]



KeyboardInterrupt



In [12]:
print("omega lul")

omega lul


In [14]:
FILENAME = "NoK4_0_1_2"

In [ ]:
%%time
BIG_NUM=3
SAVED = []
print("COLORED", COLORSS)
print('to check:', len(k_6), len(k_6))
bruteforce_nolimits_nok4_short(k_6[:80], kv, [], k_2+k_6[80:], k_4, k2s, kl3s, BIG_NUM, 40, 10, 6)
print("AFTER ALL-COLORED", COLORSS)

COLORED 2345
to check: 150 150
[[0, 0], [40, 78], [0, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], [32, 77], [1, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], [23, 75], [3, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], [10, 72], [6, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], [8, 71], [7, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], [5, 70], [8, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], [1, 69], [9, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], [67, 69], [9, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], [65, 68], [10, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], [63, 66], [12, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], [53, 64], [14, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], [41, 63], [15, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], [30, 62], [16, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], [20, 61], [17, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], [15, 60], [18, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], [10, 59], [19, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], [7, 58], [20, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], [12, 57], [21, 79], [0, 80], [0, 0], [0, 0]]
[[0, 0], 

[[0, 0], [46, 52], [23, 76], [3, 80], [0, 0], [0, 0]]
[[0, 0], [47, 51], [24, 76], [3, 80], [0, 0], [0, 0]]
[[0, 0], [1, 49], [26, 76], [3, 80], [0, 0], [0, 0]]
[[0, 0], [9, 48], [27, 76], [3, 80], [0, 0], [0, 0]]
[[0, 0], [17, 47], [28, 76], [3, 80], [0, 0], [0, 0]]
[[0, 0], [26, 46], [29, 76], [3, 80], [0, 0], [0, 0]]
[[0, 0], [38, 45], [30, 76], [3, 80], [0, 0], [0, 0]]
[[0, 0], [20, 43], [32, 76], [3, 80], [0, 0], [0, 0]]
[[0, 0], [38, 42], [33, 76], [3, 80], [0, 0], [0, 0]]
[[0, 0], [27, 40], [35, 76], [3, 80], [0, 0], [0, 0]]
[[0, 0], [20, 38], [37, 76], [3, 80], [0, 0], [0, 0]]
[[0, 0], [10, 36], [39, 76], [3, 80], [0, 0], [0, 0]]
[[0, 0], [25, 35], [40, 76], [3, 80], [0, 0], [0, 0]]
[[0, 0], [8, 33], [42, 76], [3, 80], [0, 0], [0, 0]]
[[0, 0], [27, 32], [43, 76], [3, 80], [0, 0], [0, 0]]
[[0, 0], [15, 30], [45, 76], [3, 80], [0, 0], [0, 0]]
[[0, 0], [6, 28], [47, 76], [3, 80], [0, 0], [0, 0]]
[[0, 0], [1, 26], [49, 76], [3, 80], [0, 0], [0, 0]]
[[0, 0], [0, 24], [51, 76], [3, 8

[[0, 0], [26, 31], [40, 72], [7, 80], [0, 0], [0, 0]]
[[0, 0], [27, 29], [42, 72], [7, 80], [0, 0], [0, 0]]
[[0, 0], [8, 26], [45, 72], [7, 80], [0, 0], [0, 0]]
[[0, 0], [22, 24], [47, 72], [7, 80], [0, 0], [0, 0]]
[[0, 0], [1, 20], [51, 72], [7, 80], [0, 0], [0, 0]]
[[0, 0], [0, 10], [61, 72], [7, 80], [0, 0], [0, 0]]
[[0, 0], [1, 3], [68, 72], [7, 80], [0, 0], [0, 0]]
[[0, 0], [16, 69], [1, 71], [8, 80], [0, 0], [0, 0]]
[[0, 0], [36, 67], [3, 71], [8, 80], [0, 0], [0, 0]]
[[0, 0], [58, 66], [4, 71], [8, 80], [0, 0], [0, 0]]
[[0, 0], [25, 63], [7, 71], [8, 80], [0, 0], [0, 0]]
[[0, 0], [32, 61], [9, 71], [8, 80], [0, 0], [0, 0]]
[[0, 0], [38, 60], [10, 71], [8, 80], [0, 0], [0, 0]]
[[0, 0], [50, 59], [11, 71], [8, 80], [0, 0], [0, 0]]
[[0, 0], [14, 54], [16, 71], [8, 80], [0, 0], [0, 0]]
[[0, 0], [32, 51], [19, 71], [8, 80], [0, 0], [0, 0]]
[[0, 0], [0, 49], [21, 71], [8, 80], [0, 0], [0, 0]]
[[0, 0], [16, 48], [22, 71], [8, 80], [0, 0], [0, 0]]
[[0, 0], [28, 47], [23, 71], [8, 80], [

[[0, 0], [8, 34], [31, 66], [13, 80], [0, 0], [0, 0]]
[[0, 0], [14, 32], [33, 66], [13, 80], [0, 0], [0, 0]]
[[0, 0], [17, 30], [35, 66], [13, 80], [0, 0], [0, 0]]
[[0, 0], [20, 28], [37, 66], [13, 80], [0, 0], [0, 0]]
[[0, 0], [21, 26], [39, 66], [13, 80], [0, 0], [0, 0]]
[[0, 0], [2, 23], [42, 66], [13, 80], [0, 0], [0, 0]]
[[0, 0], [16, 21], [44, 66], [13, 80], [0, 0], [0, 0]]
[[0, 0], [3, 14], [51, 66], [13, 80], [0, 0], [0, 0]]
[[0, 0], [3, 10], [55, 66], [13, 80], [0, 0], [0, 0]]
[0, 945, 910] [414, 438, 442]
[[0, 0], [1, 2], [63, 66], [13, 80], [0, 0], [0, 0]]
[[0, 0], [22, 63], [1, 65], [14, 80], [0, 0], [0, 0]]
[[0, 0], [33, 62], [2, 65], [14, 80], [0, 0], [0, 0]]
[[0, 0], [53, 60], [4, 65], [14, 80], [0, 0], [0, 0]]
[[0, 0], [20, 58], [6, 65], [14, 80], [0, 0], [0, 0]]
[[0, 0], [35, 57], [7, 65], [14, 80], [0, 0], [0, 0]]
[[0, 0], [47, 56], [8, 65], [14, 80], [0, 0], [0, 0]]
[[0, 0], [4, 48], [16, 65], [14, 80], [0, 0], [0, 0]]
[[0, 0], [21, 47], [17, 65], [14, 80], [0, 0], [

[[0, 0], [49, 55], [3, 59], [20, 80], [0, 0], [0, 0]]
[[0, 0], [24, 51], [7, 59], [20, 80], [0, 0], [0, 0]]
[[0, 0], [47, 50], [8, 59], [20, 80], [0, 0], [0, 0]]
[[0, 0], [25, 44], [14, 59], [20, 80], [0, 0], [0, 0]]
[[0, 0], [11, 42], [16, 59], [20, 80], [0, 0], [0, 0]]
[[0, 0], [32, 41], [17, 59], [20, 80], [0, 0], [0, 0]]
[[0, 0], [17, 39], [19, 59], [20, 80], [0, 0], [0, 0]]
[[0, 0], [11, 37], [21, 59], [20, 80], [0, 0], [0, 0]]
[[0, 0], [9, 35], [23, 59], [20, 80], [0, 0], [0, 0]]
[[0, 0], [11, 33], [25, 59], [20, 80], [0, 0], [0, 0]]
[[0, 0], [25, 31], [27, 59], [20, 80], [0, 0], [0, 0]]
[[0, 0], [10, 25], [33, 59], [20, 80], [0, 0], [0, 0]]
[[0, 0], [17, 23], [35, 59], [20, 80], [0, 0], [0, 0]]
[[0, 0], [9, 19], [39, 59], [20, 80], [0, 0], [0, 0]]
[[0, 0], [15, 16], [42, 59], [20, 80], [0, 0], [0, 0]]
[[0, 0], [0, 10], [48, 59], [20, 80], [0, 0], [0, 0]]
[[0, 0], [35, 57], [0, 58], [21, 80], [0, 0], [0, 0]]
[[0, 0], [14, 53], [4, 58], [21, 80], [0, 0], [0, 0]]
[[0, 0], [36, 52],

[[0, 0], [34, 46], [1, 48], [31, 80], [0, 0], [0, 0]]
[[0, 0], [24, 41], [6, 48], [31, 80], [0, 0], [0, 0]]
[[0, 0], [26, 36], [11, 48], [31, 80], [0, 0], [0, 0]]
[[0, 0], [5, 32], [15, 48], [31, 80], [0, 0], [0, 0]]
[[0, 0], [19, 30], [17, 48], [31, 80], [0, 0], [0, 0]]
[[0, 0], [4, 24], [23, 48], [31, 80], [0, 0], [0, 0]]
[[0, 0], [17, 22], [25, 48], [31, 80], [0, 0], [0, 0]]
[[0, 0], [17, 18], [29, 48], [31, 80], [0, 0], [0, 0]]
[[0, 0], [6, 12], [35, 48], [31, 80], [0, 0], [0, 0]]
[[0, 0], [13, 45], [1, 47], [32, 80], [0, 0], [0, 0]]
[[0, 0], [40, 41], [5, 47], [32, 80], [0, 0], [0, 0]]
[[0, 0], [9, 35], [11, 47], [32, 80], [0, 0], [0, 0]]
[[0, 0], [19, 32], [14, 47], [32, 80], [0, 0], [0, 0]]
[[0, 0], [6, 29], [17, 47], [32, 80], [0, 0], [0, 0]]
[[0, 0], [17, 24], [22, 47], [32, 80], [0, 0], [0, 0]]
[[0, 0], [10, 21], [25, 47], [32, 80], [0, 0], [0, 0]]
[[0, 0], [15, 17], [29, 47], [32, 80], [0, 0], [0, 0]]
[[0, 0], [10, 11], [35, 47], [32, 80], [0, 0], [0, 0]]
[[0, 0], [28, 43], 

[[0, 0], [29, 31], [4, 36], [43, 80], [0, 0], [0, 0]]
[0, 945, 910] [670, 686, 693]
[0, 945, 910] [670, 686, 697]
[0, 945, 910] [670, 686, 663]
[[0, 0], [21, 29], [6, 36], [43, 80], [0, 0], [0, 0]]
[0, 945, 910] [670, 686, 667]
[0, 945, 910] [670, 686, 700]
[0, 945, 910] [670, 691, 663]
[0, 945, 910] [670, 691, 669]
[0, 945, 910] [670, 691, 683]
[0, 945, 910] [670, 691, 694]
[0, 945, 910] [670, 691, 698]
[[0, 0], [14, 27], [8, 36], [43, 80], [0, 0], [0, 0]]
[0, 945, 910] [670, 697, 685]
[0, 945, 910] [670, 697, 698]
[[0, 0], [11, 25], [10, 36], [43, 80], [0, 0], [0, 0]]
[0, 945, 910] [670, 711, 663]
[0, 945, 910] [670, 715, 667]
[[0, 0], [18, 23], [12, 36], [43, 80], [0, 0], [0, 0]]
[[0, 0], [4, 20], [15, 36], [43, 80], [0, 0], [0, 0]]
[[0, 0], [16, 18], [17, 36], [43, 80], [0, 0], [0, 0]]
[[0, 0], [4, 14], [21, 36], [43, 80], [0, 0], [0, 0]]
[0, 945, 910] [670, 413, 663]
[0, 945, 910] [670, 413, 667]
[0, 945, 910] [670, 413, 669]
[0, 945, 910] [670, 438, 694]
[0, 945, 910] [670, 438, 

In [24]:
1+1

2

In [ ]:
print(SAVED)

In [ ]:
%%time
BIG_NUM=2
print("COLORED", COLORSS)
print('to check:', len(k_6), len(k_6))
bruteforce_nolimits_nok4_short(k_6[:80], kv, [], k_2+k_6[80:], k_4, k2s, kl3s, BIG_NUM, 20, 10, 6)
print("AFTER ALL-COLORED", COLORSS)

In [ ]:
%%time
BIG_NUM=1
print("COLORED", COLORSS)
print('to check:', len(k_6), len(k_6))
bruteforce_nolimits_nok4_short(k_6[:80], kv, [], k_2+k_6[80:], k_4, k2s, kl3s, BIG_NUM, 20, 10, 6)
print("AFTER ALL-COLORED", COLORSS)

In [31]:
print(SAVED)

[([0, 945, 910], [175, 686, 685]), ([0, 945, 910], [414, 438, 442]), ([0, 945, 910], [430, 686, 698]), ([0, 945, 910], [430, 686, 700]), ([0, 945, 910], [437, 413, 438]), ([0, 945, 910], [469, 476, 413]), ([0, 945, 910], [504, 442, 444]), ([0, 945, 910], [543, 670, 667]), ([0, 945, 910], [559, 655, 669]), ([0, 945, 910], [559, 686, 679]), ([0, 945, 910], [573, 693, 685]), ([0, 945, 910], [655, 670, 686]), ([0, 945, 910], [655, 686, 691]), ([0, 945, 910], [655, 686, 697]), ([0, 945, 910], [655, 686, 667]), ([0, 945, 910], [655, 686, 683]), ([0, 945, 910], [655, 691, 697]), ([0, 945, 910], [655, 691, 667]), ([0, 945, 910], [655, 693, 667]), ([0, 945, 910], [655, 697, 679]), ([0, 945, 910], [655, 715, 679]), ([0, 945, 910], [655, 725, 663]), ([0, 945, 910], [655, 407, 669]), ([0, 945, 910], [655, 663, 667]), ([0, 945, 910], [655, 663, 679]), ([0, 945, 910], [655, 663, 685]), ([0, 945, 910], [655, 667, 669]), ([0, 945, 910], [655, 667, 679]), ([0, 945, 910], [655, 667, 683]), ([0, 945, 910

In [ ]:
1+1

In [32]:
lss = [([0, 945, 910], [175, 686, 685]), ([0, 945, 910], [414, 438, 442]), ([0, 945, 910], [430, 686, 698]), ([0, 945, 910], [430, 686, 700]), ([0, 945, 910], [437, 413, 438]), ([0, 945, 910], [469, 476, 413]), ([0, 945, 910], [504, 442, 444]), ([0, 945, 910], [543, 670, 667]), ([0, 945, 910], [559, 655, 669]), ([0, 945, 910], [559, 686, 679]), ([0, 945, 910], [573, 693, 685]), ([0, 945, 910], [655, 670, 686]), ([0, 945, 910], [655, 686, 691]), ([0, 945, 910], [655, 686, 697]), ([0, 945, 910], [655, 686, 667]), ([0, 945, 910], [655, 686, 683]), ([0, 945, 910], [655, 691, 697]), ([0, 945, 910], [655, 691, 667]), ([0, 945, 910], [655, 693, 667]), ([0, 945, 910], [655, 697, 679]), ([0, 945, 910], [655, 715, 679]), ([0, 945, 910], [655, 725, 663]), ([0, 945, 910], [655, 407, 669]), ([0, 945, 910], [655, 663, 667]), ([0, 945, 910], [655, 663, 679]), ([0, 945, 910], [655, 663, 685]), ([0, 945, 910], [655, 667, 669]), ([0, 945, 910], [655, 667, 679]), ([0, 945, 910], [655, 667, 683]), ([0, 945, 910], [655, 669, 683]), ([0, 945, 910], [655, 669, 685]), ([0, 945, 910], [655, 679, 683]), ([0, 945, 910], [670, 686, 693]), ([0, 945, 910], [670, 686, 697]), ([0, 945, 910], [670, 686, 663]), ([0, 945, 910], [670, 686, 667]), ([0, 945, 910], [670, 686, 700]), ([0, 945, 910], [670, 691, 663]), ([0, 945, 910], [670, 691, 669]), ([0, 945, 910], [670, 691, 683]), ([0, 945, 910], [670, 691, 694]), ([0, 945, 910], [670, 691, 698]), ([0, 945, 910], [670, 697, 685]), ([0, 945, 910], [670, 697, 698]), ([0, 945, 910], [670, 711, 663]), ([0, 945, 910], [670, 715, 667]), ([0, 945, 910], [670, 413, 663]), ([0, 945, 910], [670, 413, 667]), ([0, 945, 910], [670, 413, 669]), ([0, 945, 910], [670, 438, 694]), ([0, 945, 910], [670, 438, 698]), ([0, 945, 910], [670, 663, 669]), ([0, 945, 910], [670, 663, 679]), ([0, 945, 910], [670, 663, 694]), ([0, 945, 910], [670, 663, 700]), ([0, 945, 910], [670, 667, 698]), ([0, 945, 910], [670, 669, 694]), ([0, 945, 910], [670, 669, 700]), ([0, 945, 910], [670, 698, 700]), ([0, 945, 910], [686, 691, 693]), ([0, 945, 910], [686, 691, 663]), ([0, 945, 910], [686, 691, 679]), ([0, 945, 910], [686, 691, 683]), ([0, 945, 910], [686, 691, 700]), ([0, 945, 910], [686, 693, 697]), ([0, 945, 910], [686, 693, 679]), ([0, 945, 910], [686, 693, 685]), ([0, 945, 910], [686, 697, 683]), ([0, 945, 910], [686, 697, 685]), ([0, 945, 910], [686, 746, 683]), ([0, 945, 910], [686, 438, 700]), ([0, 945, 910], [686, 442, 698]), ([0, 945, 910], [686, 442, 700]), ([0, 945, 910], [686, 663, 679]), ([0, 945, 910], [686, 663, 700]), ([0, 945, 910], [686, 679, 683]), ([0, 945, 910], [686, 679, 685]), ([0, 945, 910], [686, 679, 698]), ([0, 945, 910], [686, 683, 685]), ([0, 945, 910], [686, 685, 698]), ([0, 945, 910], [686, 698, 700]), ([0, 945, 910], [691, 693, 697]), ([0, 945, 910], [691, 693, 694]), ([0, 945, 910], [691, 697, 663]), ([0, 945, 910], [691, 697, 667]), ([0, 945, 910], [691, 697, 685]), ([0, 945, 910], [691, 715, 683]), ([0, 945, 910], [691, 730, 667]), ([0, 945, 910], [691, 442, 683]), ([0, 945, 910], [691, 663, 669]), ([0, 945, 910], [691, 663, 685]), ([0, 945, 910], [691, 663, 694]), ([0, 945, 910], [691, 667, 683]), ([0, 945, 910], [691, 694, 698]), ([0, 945, 910], [691, 698, 700]), ([0, 945, 910], [693, 697, 669]), ([0, 945, 910], [693, 697, 700]), ([0, 945, 910], [693, 711, 663]), ([0, 945, 910], [693, 725, 679]), ([0, 945, 910], [693, 725, 694]), ([0, 945, 910], [693, 741, 663]), ([0, 945, 910], [693, 741, 700]), ([0, 945, 910], [693, 745, 685]), ([0, 945, 910], [693, 413, 663]), ([0, 945, 910], [693, 423, 694]), ([0, 945, 910], [693, 429, 679]), ([0, 945, 910], [693, 438, 694]), ([0, 945, 910], [693, 663, 679]), ([0, 945, 910], [693, 663, 694]), ([0, 945, 910], [693, 663, 700]), ([0, 945, 910], [693, 685, 700]), ([0, 945, 910], [693, 694, 700]), ([0, 945, 910], [697, 427, 667]), ([0, 945, 910], [697, 427, 698]), ([0, 945, 910], [697, 444, 700]), ([0, 945, 910], [697, 663, 700]), ([0, 945, 910], [697, 669, 685]), ([0, 945, 910], [697, 669, 698]), ([0, 945, 910], [697, 683, 698]), ([0, 945, 910], [697, 694, 700]), ([0, 945, 910], [723, 726, 663]), ([0, 945, 910], [411, 667, 683]), ([0, 945, 910], [438, 442, 700]), ([0, 945, 910], [438, 694, 698]), ([0, 945, 910], [444, 669, 700]), ([0, 945, 910], [444, 685, 700]), ([0, 945, 910], [667, 683, 685]), ([0, 945, 910], [667, 694, 698])]
print(len(lss))

128


In [39]:
len(SAVED)

0

In [40]:
SAVED = [([0, 945, 910], [175, 686, 685]), ([0, 945, 910], [414, 438, 442]), ([0, 945, 910], [430, 686, 698]), ([0, 945, 910], [430, 686, 700]), ([0, 945, 910], [437, 413, 438]), ([0, 945, 910], [469, 476, 413]), ([0, 945, 910], [504, 442, 444]), ([0, 945, 910], [543, 670, 667]), ([0, 945, 910], [559, 655, 669]), ([0, 945, 910], [559, 686, 679]), ([0, 945, 910], [573, 693, 685]), ([0, 945, 910], [655, 670, 686]), ([0, 945, 910], [655, 686, 691]), ([0, 945, 910], [655, 686, 697]), ([0, 945, 910], [655, 686, 667]), ([0, 945, 910], [655, 686, 683]), ([0, 945, 910], [655, 691, 697]), ([0, 945, 910], [655, 691, 667]), ([0, 945, 910], [655, 693, 667]), ([0, 945, 910], [655, 697, 679]), ([0, 945, 910], [655, 715, 679]), ([0, 945, 910], [655, 725, 663]), ([0, 945, 910], [655, 407, 669]), ([0, 945, 910], [655, 663, 667]), ([0, 945, 910], [655, 663, 679]), ([0, 945, 910], [655, 663, 685]), ([0, 945, 910], [655, 667, 669]), ([0, 945, 910], [655, 667, 679]), ([0, 945, 910], [655, 667, 683]), ([0, 945, 910], [655, 669, 683]), ([0, 945, 910], [655, 669, 685]), ([0, 945, 910], [655, 679, 683]), ([0, 945, 910], [670, 686, 693]), ([0, 945, 910], [670, 686, 697]), ([0, 945, 910], [670, 686, 663]), ([0, 945, 910], [670, 686, 667]), ([0, 945, 910], [670, 686, 700]), ([0, 945, 910], [670, 691, 663]), ([0, 945, 910], [670, 691, 669]), ([0, 945, 910], [670, 691, 683]), ([0, 945, 910], [670, 691, 694]), ([0, 945, 910], [670, 691, 698]), ([0, 945, 910], [670, 697, 685]), ([0, 945, 910], [670, 697, 698]), ([0, 945, 910], [670, 711, 663]), ([0, 945, 910], [670, 715, 667]), ([0, 945, 910], [670, 413, 663]), ([0, 945, 910], [670, 413, 667]), ([0, 945, 910], [670, 413, 669]), ([0, 945, 910], [670, 438, 694]), ([0, 945, 910], [670, 438, 698]), ([0, 945, 910], [670, 663, 669]), ([0, 945, 910], [670, 663, 679]), ([0, 945, 910], [670, 663, 694]), ([0, 945, 910], [670, 663, 700]), ([0, 945, 910], [670, 667, 698]), ([0, 945, 910], [670, 669, 694]), ([0, 945, 910], [670, 669, 700]), ([0, 945, 910], [670, 698, 700]), ([0, 945, 910], [686, 691, 693]), ([0, 945, 910], [686, 691, 663]), ([0, 945, 910], [686, 691, 679]), ([0, 945, 910], [686, 691, 683]), ([0, 945, 910], [686, 691, 700]), ([0, 945, 910], [686, 693, 697]), ([0, 945, 910], [686, 693, 679]), ([0, 945, 910], [686, 693, 685]), ([0, 945, 910], [686, 697, 683]), ([0, 945, 910], [686, 697, 685]), ([0, 945, 910], [686, 746, 683]), ([0, 945, 910], [686, 438, 700]), ([0, 945, 910], [686, 442, 698]), ([0, 945, 910], [686, 442, 700]), ([0, 945, 910], [686, 663, 679]), ([0, 945, 910], [686, 663, 700]), ([0, 945, 910], [686, 679, 683]), ([0, 945, 910], [686, 679, 685]), ([0, 945, 910], [686, 679, 698]), ([0, 945, 910], [686, 683, 685]), ([0, 945, 910], [686, 685, 698]), ([0, 945, 910], [686, 698, 700]), ([0, 945, 910], [691, 693, 697]), ([0, 945, 910], [691, 693, 694]), ([0, 945, 910], [691, 697, 663]), ([0, 945, 910], [691, 697, 667]), ([0, 945, 910], [691, 697, 685]), ([0, 945, 910], [691, 715, 683]), ([0, 945, 910], [691, 730, 667]), ([0, 945, 910], [691, 442, 683]), ([0, 945, 910], [691, 663, 669]), ([0, 945, 910], [691, 663, 685]), ([0, 945, 910], [691, 663, 694]), ([0, 945, 910], [691, 667, 683]), ([0, 945, 910], [691, 694, 698]), ([0, 945, 910], [691, 698, 700]), ([0, 945, 910], [693, 697, 669]), ([0, 945, 910], [693, 697, 700]), ([0, 945, 910], [693, 711, 663]), ([0, 945, 910], [693, 725, 679]), ([0, 945, 910], [693, 725, 694]), ([0, 945, 910], [693, 741, 663]), ([0, 945, 910], [693, 741, 700]), ([0, 945, 910], [693, 745, 685]), ([0, 945, 910], [693, 413, 663]), ([0, 945, 910], [693, 423, 694]), ([0, 945, 910], [693, 429, 679]), ([0, 945, 910], [693, 438, 694]), ([0, 945, 910], [693, 663, 679]), ([0, 945, 910], [693, 663, 694]), ([0, 945, 910], [693, 663, 700]), ([0, 945, 910], [693, 685, 700]), ([0, 945, 910], [693, 694, 700]), ([0, 945, 910], [697, 427, 667]), ([0, 945, 910], [697, 427, 698]), ([0, 945, 910], [697, 444, 700]), ([0, 945, 910], [697, 663, 700]), ([0, 945, 910], [697, 669, 685]), ([0, 945, 910], [697, 669, 698]), ([0, 945, 910], [697, 683, 698]), ([0, 945, 910], [697, 694, 700]), ([0, 945, 910], [723, 726, 663]), ([0, 945, 910], [411, 667, 683]), ([0, 945, 910], [438, 442, 700]), ([0, 945, 910], [438, 694, 698]), ([0, 945, 910], [444, 669, 700]), ([0, 945, 910], [444, 685, 700]), ([0, 945, 910], [667, 683, 685]), ([0, 945, 910], [667, 694, 698]), ([0, 945, 910], [159, 543]), ([0, 945, 910], [159, 655]), ([0, 945, 910], [159, 670]), ([0, 945, 910], [159, 686]), ([0, 945, 910], [159, 691]), ([0, 945, 910], [159, 693]), ([0, 945, 910], [159, 697]), ([0, 945, 910], [159, 411]), ([0, 945, 910], [159, 663]), ([0, 945, 910], [159, 667]), ([0, 945, 910], [159, 669]), ([0, 945, 910], [159, 683]), ([0, 945, 910], [175, 430]), ([0, 945, 910], [175, 717]), ([0, 945, 910], [175, 423]), ([0, 945, 910], [175, 679]), ([0, 945, 910], [175, 683]), ([0, 945, 910], [175, 685]), ([0, 945, 910], [183, 414]), ([0, 945, 910], [183, 435]), ([0, 945, 910], [183, 437]), ([0, 945, 910], [183, 691]), ([0, 945, 910], [183, 693]), ([0, 945, 910], [183, 407]), ([0, 945, 910], [183, 423]), ([0, 945, 910], [183, 663]), ([0, 945, 910], [183, 698]), ([0, 945, 910], [187, 655]), ([0, 945, 910], [187, 691]), ([0, 945, 910], [187, 411]), ([0, 945, 910], [187, 427]), ([0, 945, 910], [187, 438]), ([0, 945, 910], [187, 663]), ([0, 945, 910], [187, 667]), ([0, 945, 910], [187, 683]), ([0, 945, 910], [187, 698]), ([0, 945, 910], [189, 317]), ([0, 945, 910], [189, 437]), ([0, 945, 910], [189, 441]), ([0, 945, 910], [189, 686]), ([0, 945, 910], [189, 693]), ([0, 945, 910], [189, 413]), ([0, 945, 910], [189, 444]), ([0, 945, 910], [189, 685]), ([0, 945, 910], [190, 318]), ([0, 945, 910], [190, 430]), ([0, 945, 910], [190, 686]), ([0, 945, 910], [190, 691]), ([0, 945, 910], [190, 693]), ([0, 945, 910], [190, 438]), ([0, 945, 910], [190, 442]), ([0, 945, 910], [190, 444]), ([0, 945, 910], [190, 700]), ([0, 945, 910], [287, 399]), ([0, 945, 910], [287, 430]), ([0, 945, 910], [287, 441]), ([0, 945, 910], [287, 670]), ([0, 945, 910], [287, 407]), ([0, 945, 910], [287, 442]), ([0, 945, 910], [287, 663]), ([0, 945, 910], [287, 669]), ([0, 945, 910], [303, 399]), ([0, 945, 910], [303, 427]), ([0, 945, 910], [303, 429]), ([0, 945, 910], [303, 683]), ([0, 945, 910], [311, 437]), ([0, 945, 910], [311, 423]), ([0, 945, 910], [311, 694]), ([0, 945, 910], [315, 414]), ([0, 945, 910], [315, 435]), ([0, 945, 910], [315, 441]), ([0, 945, 910], [315, 691]), ([0, 945, 910], [315, 411]), ([0, 945, 910], [315, 427]), ([0, 945, 910], [315, 442]), ([0, 945, 910], [315, 683]), ([0, 945, 910], [315, 698]), ([0, 945, 910], [317, 437]), ([0, 945, 910], [317, 413]), ([0, 945, 910], [317, 438]), ([0, 945, 910], [317, 444]), ([0, 945, 910], [317, 685]), ([0, 945, 910], [318, 414]), ([0, 945, 910], [318, 430]), ([0, 945, 910], [318, 435]), ([0, 945, 910], [318, 574]), ([0, 945, 910], [318, 413]), ([0, 945, 910], [318, 423]), ([0, 945, 910], [318, 427]), ([0, 945, 910], [318, 438]), ([0, 945, 910], [318, 442]), ([0, 945, 910], [318, 444]), ([0, 945, 910], [318, 694]), ([0, 945, 910], [318, 698]), ([0, 945, 910], [318, 700]), ([0, 945, 910], [399, 414]), ([0, 945, 910], [399, 430]), ([0, 945, 910], [399, 435]), ([0, 945, 910], [399, 437]), ([0, 945, 910], [399, 441]), ([0, 945, 910], [399, 459]), ([0, 945, 910], [399, 461]), ([0, 945, 910], [399, 485]), ([0, 945, 910], [399, 670]), ([0, 945, 910], [399, 686]), ([0, 945, 910], [399, 717]), ([0, 945, 910], [399, 407]), ([0, 945, 910], [399, 411]), ([0, 945, 910], [399, 427]), ([0, 945, 910], [399, 429]), ([0, 945, 910], [399, 438]), ([0, 945, 910], [399, 442]), ([0, 945, 910], [399, 679]), ([0, 945, 910], [399, 683]), ([0, 945, 910], [414, 430]), ([0, 945, 910], [414, 435]), ([0, 945, 910], [414, 437]), ([0, 945, 910], [414, 441]), ([0, 945, 910], [414, 455]), ([0, 945, 910], [414, 467]), ([0, 945, 910], [414, 469]), ([0, 945, 910], [414, 470]), ([0, 945, 910], [414, 474]), ([0, 945, 910], [414, 476]), ([0, 945, 910], [414, 486]), ([0, 945, 910], [414, 490]), ([0, 945, 910], [414, 498]), ([0, 945, 910], [414, 500]), ([0, 945, 910], [414, 504]), ([0, 945, 910], [414, 543]), ([0, 945, 910], [414, 655]), ([0, 945, 910], [414, 670]), ([0, 945, 910], [414, 686]), ([0, 945, 910], [414, 730]), ([0, 945, 910], [414, 732]), ([0, 945, 910], [414, 407]), ([0, 945, 910], [414, 411]), ([0, 945, 910], [414, 413]), ([0, 945, 910], [414, 423]), ([0, 945, 910], [414, 429]), ([0, 945, 910], [414, 438]), ([0, 945, 910], [414, 442]), ([0, 945, 910], [414, 444]), ([0, 945, 910], [414, 663]), ([0, 945, 910], [414, 667]), ([0, 945, 910], [414, 669]), ([0, 945, 910], [414, 694]), ([0, 945, 910], [414, 698]), ([0, 945, 910], [414, 700]), ([0, 945, 910], [430, 467]), ([0, 945, 910], [430, 435]), ([0, 945, 910], [430, 459]), ([0, 945, 910], [430, 461]), ([0, 945, 910], [430, 470]), ([0, 945, 910], [430, 476]), ([0, 945, 910], [430, 483]), ([0, 945, 910], [430, 485]), ([0, 945, 910], [430, 486]), ([0, 945, 910], [430, 489]), ([0, 945, 910], [430, 490]), ([0, 945, 910], [430, 492]), ([0, 945, 910], [430, 498]), ([0, 945, 910], [430, 500]), ([0, 945, 910], [430, 655]), ([0, 945, 910], [430, 670]), ([0, 945, 910], [430, 686]), ([0, 945, 910], [430, 742]), ([0, 945, 910], [430, 411]), ([0, 945, 910], [430, 413]), ([0, 945, 910], [430, 423]), ([0, 945, 910], [430, 427]), ([0, 945, 910], [430, 429]), ([0, 945, 910], [430, 438]), ([0, 945, 910], [430, 442]), ([0, 945, 910], [430, 444]), ([0, 945, 910], [430, 679]), ([0, 945, 910], [430, 683]), ([0, 945, 910], [430, 685]), ([0, 945, 910], [430, 694]), ([0, 945, 910], [430, 700]), ([0, 945, 910], [435, 437]), ([0, 945, 910], [435, 467]), ([0, 945, 910], [435, 470]), ([0, 945, 910], [435, 474]), ([0, 945, 910], [435, 483]), ([0, 945, 910], [435, 500]), ([0, 945, 910], [435, 693]), ([0, 945, 910], [435, 697]), ([0, 945, 910], [435, 723]), ([0, 945, 910], [435, 739]), ([0, 945, 910], [435, 407]), ([0, 945, 910], [435, 411]), ([0, 945, 910], [435, 413]), ([0, 945, 910], [435, 423]), ([0, 945, 910], [435, 427]), ([0, 945, 910], [435, 438]), ([0, 945, 910], [435, 442]), ([0, 945, 910], [435, 444]), ([0, 945, 910], [435, 667]), ([0, 945, 910], [435, 679]), ([0, 945, 910], [435, 683]), ([0, 945, 910], [435, 694]), ([0, 945, 910], [435, 698]), ([0, 945, 910], [437, 686]), ([0, 945, 910], [437, 441]), ([0, 945, 910], [437, 455]), ([0, 945, 910], [437, 461]), ([0, 945, 910], [437, 476]), ([0, 945, 910], [437, 485]), ([0, 945, 910], [437, 486]), ([0, 945, 910], [437, 500]), ([0, 945, 910], [437, 691]), ([0, 945, 910], [437, 693]), ([0, 945, 910], [437, 697]), ([0, 945, 910], [437, 741]), ([0, 945, 910], [437, 407]), ([0, 945, 910], [437, 411]), ([0, 945, 910], [437, 413]), ([0, 945, 910], [437, 423]), ([0, 945, 910], [437, 427]), ([0, 945, 910], [437, 429]), ([0, 945, 910], [437, 438]), ([0, 945, 910], [437, 442]), ([0, 945, 910], [437, 444]), ([0, 945, 910], [437, 669]), ([0, 945, 910], [437, 679]), ([0, 945, 910], [437, 685]), ([0, 945, 910], [437, 694]), ([0, 945, 910], [437, 700]), ([0, 945, 910], [441, 459]), ([0, 945, 910], [441, 461]), ([0, 945, 910], [441, 473]), ([0, 945, 910], [441, 476]), ([0, 945, 910], [441, 489]), ([0, 945, 910], [441, 490]), ([0, 945, 910], [441, 504]), ([0, 945, 910], [441, 691]), ([0, 945, 910], [441, 694]), ([0, 945, 910], [441, 697]), ([0, 945, 910], [441, 745]), ([0, 945, 910], [441, 411]), ([0, 945, 910], [441, 413]), ([0, 945, 910], [441, 427]), ([0, 945, 910], [441, 429]), ([0, 945, 910], [441, 438]), ([0, 945, 910], [441, 442]), ([0, 945, 910], [441, 444]), ([0, 945, 910], [441, 667]), ([0, 945, 910], [441, 669]), ([0, 945, 910], [441, 698]), ([0, 945, 910], [441, 700]), ([0, 945, 910], [455, 461]), ([0, 945, 910], [455, 469]), ([0, 945, 910], [455, 483]), ([0, 945, 910], [455, 485]), ([0, 945, 910], [455, 655]), ([0, 945, 910], [455, 723]), ([0, 945, 910], [455, 407]), ([0, 945, 910], [455, 423]), ([0, 945, 910], [455, 438]), ([0, 945, 910], [459, 473]), ([0, 945, 910], [459, 483]), ([0, 945, 910], [459, 655]), ([0, 945, 910], [459, 715]), ([0, 945, 910], [459, 729]), ([0, 945, 910], [459, 739]), ([0, 945, 910], [459, 407]), ([0, 945, 910], [459, 411]), ([0, 945, 910], [459, 413]), ([0, 945, 910], [459, 442]), ([0, 945, 910], [459, 683]), ([0, 945, 910], [461, 470]), ([0, 945, 910], [461, 473]), ([0, 945, 910], [461, 476]), ([0, 945, 910], [461, 485]), ([0, 945, 910], [461, 717]), ([0, 945, 910], [461, 413]), ([0, 945, 910], [461, 429]), ([0, 945, 910], [467, 670]), ([0, 945, 910], [467, 474]), ([0, 945, 910], [467, 483]), ([0, 945, 910], [467, 711]), ([0, 945, 910], [467, 407]), ([0, 945, 910], [467, 411]), ([0, 945, 910], [467, 667]), ([0, 945, 910], [469, 476]), ([0, 945, 910], [469, 693]), ([0, 945, 910], [469, 725]), ([0, 945, 910], [469, 407]), ([0, 945, 910], [469, 411]), ([0, 945, 910], [469, 413]), ([0, 945, 910], [469, 423]), ([0, 945, 910], [469, 438]), ([0, 945, 910], [470, 474]), ([0, 945, 910], [470, 476]), ([0, 945, 910], [470, 498]), ([0, 945, 910], [470, 500]), ([0, 945, 910], [470, 670]), ([0, 945, 910], [470, 730]), ([0, 945, 910], [470, 423]), ([0, 945, 910], [470, 438]), ([0, 945, 910], [470, 444]), ([0, 945, 910], [470, 694]), ([0, 945, 910], [473, 474]), ([0, 945, 910], [473, 476]), ([0, 945, 910], [473, 485]), ([0, 945, 910], [473, 504]), ([0, 945, 910], [473, 411]), ([0, 945, 910], [473, 413]), ([0, 945, 910], [473, 667]), ([0, 945, 910], [473, 669]), ([0, 945, 910], [474, 476]), ([0, 945, 910], [474, 498]), ([0, 945, 910], [474, 504]), ([0, 945, 910], [474, 411]), ([0, 945, 910], [474, 413]), ([0, 945, 910], [474, 427]), ([0, 945, 910], [474, 438]), ([0, 945, 910], [474, 442]), ([0, 945, 910], [474, 444]), ([0, 945, 910], [474, 698]), ([0, 945, 910], [476, 500]), ([0, 945, 910], [476, 504]), ([0, 945, 910], [476, 717]), ([0, 945, 910], [476, 730]), ([0, 945, 910], [476, 413]), ([0, 945, 910], [476, 429]), ([0, 945, 910], [476, 438]), ([0, 945, 910], [476, 442]), ([0, 945, 910], [476, 444]), ([0, 945, 910], [476, 700]), ([0, 945, 910], [483, 485]), ([0, 945, 910], [483, 486]), ([0, 945, 910], [483, 490]), ([0, 945, 910], [483, 498]), ([0, 945, 910], [483, 745]), ([0, 945, 910], [483, 407]), ([0, 945, 910], [483, 423]), ([0, 945, 910], [483, 429]), ([0, 945, 910], [485, 486]), ([0, 945, 910], [485, 492]), ([0, 945, 910], [485, 413]), ([0, 945, 910], [485, 423]), ([0, 945, 910], [485, 429]), ([0, 945, 910], [485, 438]), ([0, 945, 910], [485, 444]), ([0, 945, 910], [486, 492]), ([0, 945, 910], [486, 500]), ([0, 945, 910], [486, 423]), ([0, 945, 910], [486, 427]), ([0, 945, 910], [486, 438]), ([0, 945, 910], [486, 442]), ([0, 945, 910], [486, 444]), ([0, 945, 910], [486, 679]), ([0, 945, 910], [486, 694]), ([0, 945, 910], [489, 490]), ([0, 945, 910], [489, 492]), ([0, 945, 910], [489, 498]), ([0, 945, 910], [489, 500]), ([0, 945, 910], [489, 729]), ([0, 945, 910], [489, 429]), ([0, 945, 910], [489, 442]), ([0, 945, 910], [489, 444]), ([0, 945, 910], [489, 683]), ([0, 945, 910], [490, 498]), ([0, 945, 910], [490, 504]), ([0, 945, 910], [490, 715]), ([0, 945, 910], [490, 745]), ([0, 945, 910], [490, 411]), ([0, 945, 910], [490, 423]), ([0, 945, 910], [490, 427]), ([0, 945, 910], [490, 438]), ([0, 945, 910], [490, 442]), ([0, 945, 910], [490, 444]), ([0, 945, 910], [490, 683]), ([0, 945, 910], [490, 698]), ([0, 945, 910], [492, 500]), ([0, 945, 910], [492, 504]), ([0, 945, 910], [492, 413]), ([0, 945, 910], [492, 427]), ([0, 945, 910], [492, 438]), ([0, 945, 910], [492, 442]), ([0, 945, 910], [492, 444]), ([0, 945, 910], [492, 685]), ([0, 945, 910], [498, 500]), ([0, 945, 910], [498, 691]), ([0, 945, 910], [498, 411]), ([0, 945, 910], [498, 423]), ([0, 945, 910], [498, 438]), ([0, 945, 910], [498, 442]), ([0, 945, 910], [498, 444]), ([0, 945, 910], [498, 694]), ([0, 945, 910], [500, 693]), ([0, 945, 910], [500, 741]), ([0, 945, 910], [500, 413]), ([0, 945, 910], [500, 423]), ([0, 945, 910], [500, 429]), ([0, 945, 910], [500, 438]), ([0, 945, 910], [500, 442]), ([0, 945, 910], [500, 444]), ([0, 945, 910], [500, 694]), ([0, 945, 910], [500, 700]), ([0, 945, 910], [504, 670]), ([0, 945, 910], [504, 697]), ([0, 945, 910], [504, 438]), ([0, 945, 910], [504, 442]), ([0, 945, 910], [504, 444]), ([0, 945, 910], [543, 567]), ([0, 945, 910], [543, 571]), ([0, 945, 910], [543, 573]), ([0, 945, 910], [543, 574]), ([0, 945, 910], [543, 655]), ([0, 945, 910], [543, 670]), ([0, 945, 910], [543, 691]), ([0, 945, 910], [543, 730]), ([0, 945, 910], [543, 407]), ([0, 945, 910], [543, 411]), ([0, 945, 910], [543, 663]), ([0, 945, 910], [543, 667]), ([0, 945, 910], [543, 669]), ([0, 945, 910], [543, 683]), ([0, 945, 910], [559, 655]), ([0, 945, 910], [559, 670]), ([0, 945, 910], [559, 686]), ([0, 945, 910], [559, 423]), ([0, 945, 910], [559, 427]), ([0, 945, 910], [559, 663]), ([0, 945, 910], [559, 679]), ([0, 945, 910], [559, 683]), ([0, 945, 910], [559, 685]), ([0, 945, 910], [559, 700]), ([0, 945, 910], [567, 574]), ([0, 945, 910], [567, 686]), ([0, 945, 910], [567, 691]), ([0, 945, 910], [567, 693]), ([0, 945, 910], [567, 407]), ([0, 945, 910], [567, 423]), ([0, 945, 910], [567, 663]), ([0, 945, 910], [567, 669]), ([0, 945, 910], [567, 679]), ([0, 945, 910], [567, 698]), ([0, 945, 910], [571, 573]), ([0, 945, 910], [571, 574]), ([0, 945, 910], [571, 655]), ([0, 945, 910], [571, 670]), ([0, 945, 910], [571, 697]), ([0, 945, 910], [571, 427]), ([0, 945, 910], [571, 442]), ([0, 945, 910], [571, 663]), ([0, 945, 910], [571, 667]), ([0, 945, 910], [571, 683]), ([0, 945, 910], [571, 685]), ([0, 945, 910], [571, 698]), ([0, 945, 910], [571, 700]), ([0, 945, 910], [573, 574]), ([0, 945, 910], [573, 697]), ([0, 945, 910], [573, 413]), ([0, 945, 910], [573, 429]), ([0, 945, 910], [573, 669]), ([0, 945, 910], [573, 679]), ([0, 945, 910], [573, 683]), ([0, 945, 910], [573, 685]), ([0, 945, 910], [574, 670]), ([0, 945, 910], [574, 686]), ([0, 945, 910], [574, 691]), ([0, 945, 910], [574, 438]), ([0, 945, 910], [574, 444]), ([0, 945, 910], [574, 663]), ([0, 945, 910], [574, 667]), ([0, 945, 910], [574, 669]), ([0, 945, 910], [574, 679]), ([0, 945, 910], [574, 683]), ([0, 945, 910], [574, 694]), ([0, 945, 910], [574, 698]), ([0, 945, 910], [574, 700]), ([0, 945, 910], [655, 670]), ([0, 945, 910], [655, 686]), ([0, 945, 910], [655, 691]), ([0, 945, 910], [655, 693]), ([0, 945, 910], [655, 697]), ([0, 945, 910], [655, 711]), ([0, 945, 910], [655, 715]), ([0, 945, 910], [655, 717]), ([0, 945, 910], [655, 729]), ([0, 945, 910], [655, 730]), ([0, 945, 910], [655, 739]), ([0, 945, 910], [655, 741]), ([0, 945, 910], [655, 745]), ([0, 945, 910], [655, 746]), ([0, 945, 910], [655, 407]), ([0, 945, 910], [655, 411]), ([0, 945, 910], [655, 413]), ([0, 945, 910], [655, 423]), ([0, 945, 910], [655, 429]), ([0, 945, 910], [655, 663]), ([0, 945, 910], [655, 667]), ([0, 945, 910], [655, 669]), ([0, 945, 910], [655, 679]), ([0, 945, 910], [655, 683]), ([0, 945, 910], [655, 685]), ([0, 945, 910], [655, 698]), ([0, 945, 910], [670, 686]), ([0, 945, 910], [670, 691]), ([0, 945, 910], [670, 693]), ([0, 945, 910], [670, 697]), ([0, 945, 910], [670, 711]), ([0, 945, 910], [670, 715]), ([0, 945, 910], [670, 717]), ([0, 945, 910], [670, 725]), ([0, 945, 910], [670, 726]), ([0, 945, 910], [670, 729]), ([0, 945, 910], [670, 730]), ([0, 945, 910], [670, 732]), ([0, 945, 910], [670, 742]), ([0, 945, 910], [670, 407]), ([0, 945, 910], [670, 411]), ([0, 945, 910], [670, 413]), ([0, 945, 910], [670, 438]), ([0, 945, 910], [670, 442]), ([0, 945, 910], [670, 444]), ([0, 945, 910], [670, 663]), ([0, 945, 910], [670, 667]), ([0, 945, 910], [670, 669]), ([0, 945, 910], [670, 679]), ([0, 945, 910], [670, 683]), ([0, 945, 910], [670, 685]), ([0, 945, 910], [670, 694]), ([0, 945, 910], [670, 698]), ([0, 945, 910], [670, 700]), ([0, 945, 910], [686, 691]), ([0, 945, 910], [686, 693]), ([0, 945, 910], [686, 697]), ([0, 945, 910], [686, 715]), ([0, 945, 910], [686, 717]), ([0, 945, 910], [686, 726]), ([0, 945, 910], [686, 730]), ([0, 945, 910], [686, 739]), ([0, 945, 910], [686, 742]), ([0, 945, 910], [686, 745]), ([0, 945, 910], [686, 746]), ([0, 945, 910], [686, 423]), ([0, 945, 910], [686, 427]), ([0, 945, 910], [686, 438]), ([0, 945, 910], [686, 442]), ([0, 945, 910], [686, 444]), ([0, 945, 910], [686, 663]), ([0, 945, 910], [686, 667]), ([0, 945, 910], [686, 679]), ([0, 945, 910], [686, 683]), ([0, 945, 910], [686, 685]), ([0, 945, 910], [686, 694]), ([0, 945, 910], [686, 698]), ([0, 945, 910], [686, 700]), ([0, 945, 910], [691, 717]), ([0, 945, 910], [691, 693]), ([0, 945, 910], [691, 711]), ([0, 945, 910], [691, 715]), ([0, 945, 910], [691, 723]), ([0, 945, 910], [691, 730]), ([0, 945, 910], [691, 741]), ([0, 945, 910], [691, 746]), ([0, 945, 910], [691, 407]), ([0, 945, 910], [691, 423]), ([0, 945, 910], [691, 427]), ([0, 945, 910], [691, 442]), ([0, 945, 910], [691, 663]), ([0, 945, 910], [691, 667]), ([0, 945, 910], [691, 669]), ([0, 945, 910], [691, 679]), ([0, 945, 910], [691, 683]), ([0, 945, 910], [691, 685]), ([0, 945, 910], [691, 694]), ([0, 945, 910], [691, 698]), ([0, 945, 910], [691, 700]), ([0, 945, 910], [693, 746]), ([0, 945, 910], [693, 697]), ([0, 945, 910], [693, 717]), ([0, 945, 910], [693, 725]), ([0, 945, 910], [693, 726]), ([0, 945, 910], [693, 732]), ([0, 945, 910], [693, 745]), ([0, 945, 910], [693, 407]), ([0, 945, 910], [693, 413]), ([0, 945, 910], [693, 423]), ([0, 945, 910], [693, 429]), ([0, 945, 910], [693, 444]), ([0, 945, 910], [693, 663]), ([0, 945, 910], [693, 667]), ([0, 945, 910], [693, 669]), ([0, 945, 910], [693, 679]), ([0, 945, 910], [693, 683]), ([0, 945, 910], [693, 694]), ([0, 945, 910], [693, 698]), ([0, 945, 910], [693, 700]), ([0, 945, 910], [697, 423]), ([0, 945, 910], [697, 715]), ([0, 945, 910], [697, 723]), ([0, 945, 910], [697, 730]), ([0, 945, 910], [697, 732]), ([0, 945, 910], [697, 739]), ([0, 945, 910], [697, 741]), ([0, 945, 910], [697, 745]), ([0, 945, 910], [697, 411]), ([0, 945, 910], [697, 413]), ([0, 945, 910], [697, 427]), ([0, 945, 910], [697, 442]), ([0, 945, 910], [697, 444]), ([0, 945, 910], [697, 663]), ([0, 945, 910], [697, 667]), ([0, 945, 910], [697, 669]), ([0, 945, 910], [697, 679]), ([0, 945, 910], [697, 683]), ([0, 945, 910], [697, 685]), ([0, 945, 910], [697, 694]), ([0, 945, 910], [697, 698]), ([0, 945, 910], [697, 700]), ([0, 945, 910], [711, 715]), ([0, 945, 910], [711, 717]), ([0, 945, 910], [711, 725]), ([0, 945, 910], [711, 739]), ([0, 945, 910], [711, 741]), ([0, 945, 910], [711, 742]), ([0, 945, 910], [711, 745]), ([0, 945, 910], [711, 746]), ([0, 945, 910], [711, 423]), ([0, 945, 910], [711, 663]), ([0, 945, 910], [711, 667]), ([0, 945, 910], [711, 669]), ([0, 945, 910], [711, 685]), ([0, 945, 910], [711, 694]), ([0, 945, 910], [715, 717]), ([0, 945, 910], [715, 723]), ([0, 945, 910], [715, 725]), ([0, 945, 910], [715, 729]), ([0, 945, 910], [715, 730]), ([0, 945, 910], [715, 745]), ([0, 945, 910], [715, 411]), ([0, 945, 910], [715, 427]), ([0, 945, 910], [715, 663]), ([0, 945, 910], [715, 667]), ([0, 945, 910], [715, 683]), ([0, 945, 910], [715, 685]), ([0, 945, 910], [717, 725]), ([0, 945, 910], [717, 732]), ([0, 945, 910], [717, 741]), ([0, 945, 910], [717, 745]), ([0, 945, 910], [717, 413]), ([0, 945, 910], [717, 429]), ([0, 945, 910], [717, 667]), ([0, 945, 910], [717, 669]), ([0, 945, 910], [717, 679]), ([0, 945, 910], [717, 700]), ([0, 945, 910], [723, 729]), ([0, 945, 910], [723, 739]), ([0, 945, 910], [723, 407]), ([0, 945, 910], [723, 411]), ([0, 945, 910], [723, 663]), ([0, 945, 910], [723, 669]), ([0, 945, 910], [723, 683]), ([0, 945, 910], [725, 726]), ([0, 945, 910], [725, 729]), ([0, 945, 910], [725, 732]), ([0, 945, 910], [725, 741]), ([0, 945, 910], [725, 742]), ([0, 945, 910], [725, 413]), ([0, 945, 910], [725, 663]), ([0, 945, 910], [725, 667]), ([0, 945, 910], [725, 669]), ([0, 945, 910], [725, 685]), ([0, 945, 910], [725, 694]), ([0, 945, 910], [726, 732]), ([0, 945, 910], [726, 741]), ([0, 945, 910], [726, 663]), ([0, 945, 910], [726, 667]), ([0, 945, 910], [726, 679]), ([0, 945, 910], [726, 694]), ([0, 945, 910], [726, 698]), ([0, 945, 910], [729, 730]), ([0, 945, 910], [729, 411]), ([0, 945, 910], [729, 413]), ([0, 945, 910], [729, 663]), ([0, 945, 910], [729, 667]), ([0, 945, 910], [729, 669]), ([0, 945, 910], [729, 683]), ([0, 945, 910], [729, 698]), ([0, 945, 910], [729, 700]), ([0, 945, 910], [730, 732]), ([0, 945, 910], [730, 745]), ([0, 945, 910], [730, 442]), ([0, 945, 910], [730, 663]), ([0, 945, 910], [730, 667]), ([0, 945, 910], [730, 669]), ([0, 945, 910], [730, 683]), ([0, 945, 910], [730, 694]), ([0, 945, 910], [730, 700]), ([0, 945, 910], [732, 413]), ([0, 945, 910], [732, 663]), ([0, 945, 910], [732, 669]), ([0, 945, 910], [732, 685]), ([0, 945, 910], [732, 698]), ([0, 945, 910], [739, 669]), ([0, 945, 910], [739, 700]), ([0, 945, 910], [739, 746]), ([0, 945, 910], [739, 423]), ([0, 945, 910], [739, 663]), ([0, 945, 910], [739, 667]), ([0, 945, 910], [739, 683]), ([0, 945, 910], [739, 685]), ([0, 945, 910], [739, 694]), ([0, 945, 910], [741, 423]), ([0, 945, 910], [741, 663]), ([0, 945, 910], [741, 679]), ([0, 945, 910], [741, 683]), ([0, 945, 910], [741, 685]), ([0, 945, 910], [741, 694]), ([0, 945, 910], [741, 700]), ([0, 945, 910], [742, 423]), ([0, 945, 910], [742, 663]), ([0, 945, 910], [742, 679]), ([0, 945, 910], [742, 683]), ([0, 945, 910], [742, 685]), ([0, 945, 910], [742, 694]), ([0, 945, 910], [742, 700]), ([0, 945, 910], [745, 429]), ([0, 945, 910], [745, 669]), ([0, 945, 910], [745, 683]), ([0, 945, 910], [745, 698]), ([0, 945, 910], [745, 700]), ([0, 945, 910], [746, 442]), ([0, 945, 910], [746, 667]), ([0, 945, 910], [746, 683]), ([0, 945, 910], [746, 685]), ([0, 945, 910], [746, 700]), ([0, 945, 910], [407, 413]), ([0, 945, 910], [407, 423]), ([0, 945, 910], [407, 427]), ([0, 945, 910], [407, 429]), ([0, 945, 910], [407, 438]), ([0, 945, 910], [407, 444]), ([0, 945, 910], [407, 663]), ([0, 945, 910], [407, 667]), ([0, 945, 910], [407, 669]), ([0, 945, 910], [407, 679]), ([0, 945, 910], [407, 694]), ([0, 945, 910], [411, 413]), ([0, 945, 910], [411, 423]), ([0, 945, 910], [411, 427]), ([0, 945, 910], [411, 429]), ([0, 945, 910], [411, 438]), ([0, 945, 910], [411, 442]), ([0, 945, 910], [411, 444]), ([0, 945, 910], [411, 663]), ([0, 945, 910], [411, 667]), ([0, 945, 910], [411, 669]), ([0, 945, 910], [411, 683]), ([0, 945, 910], [413, 683]), ([0, 945, 910], [413, 423]), ([0, 945, 910], [413, 427]), ([0, 945, 910], [413, 429]), ([0, 945, 910], [413, 438]), ([0, 945, 910], [413, 442]), ([0, 945, 910], [413, 444]), ([0, 945, 910], [413, 663]), ([0, 945, 910], [413, 667]), ([0, 945, 910], [413, 700]), ([0, 945, 910], [423, 427]), ([0, 945, 910], [423, 429]), ([0, 945, 910], [423, 438]), ([0, 945, 910], [423, 442]), ([0, 945, 910], [423, 444]), ([0, 945, 910], [423, 679]), ([0, 945, 910], [423, 683]), ([0, 945, 910], [423, 685]), ([0, 945, 910], [423, 694]), ([0, 945, 910], [427, 429]), ([0, 945, 910], [427, 438]), ([0, 945, 910], [427, 442]), ([0, 945, 910], [427, 444]), ([0, 945, 910], [427, 667]), ([0, 945, 910], [427, 679]), ([0, 945, 910], [427, 683]), ([0, 945, 910], [427, 685]), ([0, 945, 910], [427, 698]), ([0, 945, 910], [429, 442]), ([0, 945, 910], [429, 444]), ([0, 945, 910], [429, 679]), ([0, 945, 910], [429, 683]), ([0, 945, 910], [429, 685]), ([0, 945, 910], [438, 442]), ([0, 945, 910], [438, 444]), ([0, 945, 910], [438, 663]), ([0, 945, 910], [438, 679]), ([0, 945, 910], [438, 694]), ([0, 945, 910], [438, 698]), ([0, 945, 910], [438, 700]), ([0, 945, 910], [442, 444]), ([0, 945, 910], [442, 667]), ([0, 945, 910], [442, 683]), ([0, 945, 910], [442, 698]), ([0, 945, 910], [442, 700]), ([0, 945, 910], [444, 663]), ([0, 945, 910], [444, 669]), ([0, 945, 910], [444, 685]), ([0, 945, 910], [444, 694]), ([0, 945, 910], [444, 698]), ([0, 945, 910], [444, 700]), ([0, 945, 910], [663, 667]), ([0, 945, 910], [663, 669]), ([0, 945, 910], [663, 679]), ([0, 945, 910], [663, 683]), ([0, 945, 910], [663, 685]), ([0, 945, 910], [663, 698]), ([0, 945, 910], [663, 700]), ([0, 945, 910], [667, 669]), ([0, 945, 910], [667, 679]), ([0, 945, 910], [667, 683]), ([0, 945, 910], [667, 685]), ([0, 945, 910], [667, 698]), ([0, 945, 910], [667, 700]), ([0, 945, 910], [669, 679]), ([0, 945, 910], [669, 683]), ([0, 945, 910], [669, 685]), ([0, 945, 910], [669, 694]), ([0, 945, 910], [669, 698]), ([0, 945, 910], [669, 700]), ([0, 945, 910], [679, 683]), ([0, 945, 910], [679, 685]), ([0, 945, 910], [679, 694]), ([0, 945, 910], [679, 700]), ([0, 945, 910], [683, 685]), ([0, 945, 910], [683, 698]), ([0, 945, 910], [683, 700]), ([0, 945, 910], [685, 694]), ([0, 945, 910], [685, 698]), ([0, 945, 910], [685, 700]), ([0, 945, 910], [694, 698]), ([0, 945, 910], [694, 700]), ([0, 945, 910], [698, 700]), ([0, 945, 910], [159]), ([0, 945, 910], [175]), ([0, 945, 910], [183]), ([0, 945, 910], [187]), ([0, 945, 910], [189]), ([0, 945, 910], [190]), ([0, 945, 910], [287]), ([0, 945, 910], [303]), ([0, 945, 910], [311]), ([0, 945, 910], [315]), ([0, 945, 910], [317]), ([0, 945, 910], [318]), ([0, 945, 910], [399]), ([0, 945, 910], [414]), ([0, 945, 910], [430]), ([0, 945, 910], [435]), ([0, 945, 910], [437]), ([0, 945, 910], [441]), ([0, 945, 910], [455]), ([0, 945, 910], [459]), ([0, 945, 910], [461]), ([0, 945, 910], [467]), ([0, 945, 910], [469]), ([0, 945, 910], [470]), ([0, 945, 910], [473]), ([0, 945, 910], [474]), ([0, 945, 910], [476]), ([0, 945, 910], [483]), ([0, 945, 910], [485]), ([0, 945, 910], [486]), ([0, 945, 910], [489]), ([0, 945, 910], [490]), ([0, 945, 910], [492]), ([0, 945, 910], [498]), ([0, 945, 910], [500]), ([0, 945, 910], [504]), ([0, 945, 910], [543]), ([0, 945, 910], [559]), ([0, 945, 910], [567]), ([0, 945, 910], [571]), ([0, 945, 910], [573]), ([0, 945, 910], [574]), ([0, 945, 910], [655]), ([0, 945, 910], [670]), ([0, 945, 910], [686]), ([0, 945, 910], [691]), ([0, 945, 910], [693]), ([0, 945, 910], [697]), ([0, 945, 910], [711]), ([0, 945, 910], [715]), ([0, 945, 910], [717]), ([0, 945, 910], [723]), ([0, 945, 910], [725]), ([0, 945, 910], [726]), ([0, 945, 910], [729]), ([0, 945, 910], [730]), ([0, 945, 910], [732]), ([0, 945, 910], [739]), ([0, 945, 910], [741]), ([0, 945, 910], [742]), ([0, 945, 910], [745]), ([0, 945, 910], [746]), ([0, 945, 910], [407]), ([0, 945, 910], [411]), ([0, 945, 910], [413]), ([0, 945, 910], [423]), ([0, 945, 910], [427]), ([0, 945, 910], [429]), ([0, 945, 910], [438]), ([0, 945, 910], [442]), ([0, 945, 910], [444]), ([0, 945, 910], [663]), ([0, 945, 910], [667]), ([0, 945, 910], [669]), ([0, 945, 910], [679]), ([0, 945, 910], [683]), ([0, 945, 910], [685]), ([0, 945, 910], [694]), ([0, 945, 910], [698]), ([0, 945, 910], [700])]

In [35]:
def build_blocks_by_conf(conf):
    vrtx = conf[0] + conf[1]
    lss = len(vrtx)
    k22 = []
    k33 = []
    for i in range(lss):
        for j in range(i+1, lss):
            if dist(vrtx[i], vrtx[j]) == 6:
                k22.append((vrtx[i], vrtx[j]))
    for i in range(lss):
        for j in range(i+1, lss):
            for j1 in range(j+1, lss):
                if dist(vrtx[i], vrtx[j]) == 6 and dist(vrtx[i], vrtx[j1]) == 6 and dist(vrtx[j1], vrtx[j]) == 6:
                    k33.append((vrtx[i], vrtx[j], vrtx[j1]))
    return k22, k33

In [42]:
%%time

#решаем вопрос о ровно 3 вершинах
SSAVED = []
for gr in SAVED:
    if len(gr[1]) == 3:
        k2_ss, k3_ss = build_blocks_by_conf(gr)
        grph = simple_cook_graph_no_K4(base_conf=gr[0], add_conf=gr[1], kl3s = k3_ss, k_2=k_2, k_4=k_4, k_6=[], n=10, k=6)
        brb = is_colorable(grph, 11, 20)
        if not brb:
            print(gr)
            SSAVED.append((gr[0], gr[1]))
    else:
        k2_ss, k3_ss = build_blocks_by_conf(gr)
        BIG_NUM = 3 - len(gr[1])
        bruteforce_nolimits_nok4_short(to_check=k_6[80:], base_conf=gr[0], add_conf=gr[1], k_2=k_2, k_4=k_4, k2s=k2_ss, kl3s=k2_ss, how_much=BIG_NUM, timer=20, n=10, k=6)

[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [66, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [58, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [64, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [60, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [66, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [67, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [65, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [65, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [68, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [60, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [65, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [58, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [68, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [64, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [57, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [64, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [58, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [64, 70], [0, 0], [0, 0], 

[[0, 0], [66, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [67, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [69, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [69, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [65, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [61, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [62, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [67, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [63, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [68, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [68, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [64, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [65, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [66, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [67, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [58, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [64, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [64, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [58, 70], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [68, 70], [0, 0], [0, 

[[0, 0], [10, 11], [58, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [7, 20], [49, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [7, 15], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [7, 11], [58, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [2, 19], [50, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [4, 15], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [3, 10], [59, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [1, 2], [67, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [0, 15], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [0, 10], [59, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [1, 3], [66, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [13, 16], [53, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [8, 11], [58, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [2, 4], [65, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [10, 16], [53, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [5, 11], [58, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [4, 5], [64, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [7, 16], [53, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [2, 11], [58, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [1, 

[[0, 0], [9, 19], [50, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [7, 16], [53, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [3, 12], [57, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [3, 6], [63, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [16, 22], [47, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [6, 19], [50, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [4, 16], [53, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [0, 12], [57, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [6, 7], [62, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [13, 22], [47, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [2, 19], [50, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [1, 16], [53, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [10, 13], [56, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [3, 7], [62, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [10, 22], [47, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [19, 20], [49, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [14, 17], [52, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [7, 13], [56, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [6, 8], [61, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [

KeyboardInterrupt: 

In [43]:
print(len(SSAVED), len(SAVED))

0 1046


In [44]:
print(len([([0, 945, 910], [175, 686, 685]), ([0, 945, 910], [414, 438, 442]), ([0, 945, 910], [430, 686, 698]), ([0, 945, 910], [430, 686, 700]), ([0, 945, 910], [437, 413, 438]), ([0, 945, 910], [469, 476, 413]), ([0, 945, 910], [504, 442, 444]), ([0, 945, 910], [543, 670, 667]), ([0, 945, 910], [559, 655, 669]), ([0, 945, 910], [559, 686, 679]), ([0, 945, 910], [573, 693, 685]), ([0, 945, 910], [655, 670, 686]), ([0, 945, 910], [655, 686, 691]), ([0, 945, 910], [655, 686, 697]), ([0, 945, 910], [655, 686, 667]), ([0, 945, 910], [655, 686, 683]), ([0, 945, 910], [655, 691, 697]), ([0, 945, 910], [655, 691, 667]), ([0, 945, 910], [655, 693, 667]), ([0, 945, 910], [655, 697, 679]), ([0, 945, 910], [655, 715, 679]), ([0, 945, 910], [655, 725, 663]), ([0, 945, 910], [655, 407, 669]), ([0, 945, 910], [655, 663, 667]), ([0, 945, 910], [655, 663, 679]), ([0, 945, 910], [655, 663, 685]), ([0, 945, 910], [655, 667, 669]), ([0, 945, 910], [655, 667, 679]), ([0, 945, 910], [655, 667, 683]), ([0, 945, 910], [655, 669, 683]), ([0, 945, 910], [655, 669, 685]), ([0, 945, 910], [655, 679, 683]), ([0, 945, 910], [670, 686, 693]), ([0, 945, 910], [670, 686, 697]), ([0, 945, 910], [670, 686, 663]), ([0, 945, 910], [670, 686, 667]), ([0, 945, 910], [670, 686, 700]), ([0, 945, 910], [670, 691, 663]), ([0, 945, 910], [670, 691, 669]), ([0, 945, 910], [670, 691, 683]), ([0, 945, 910], [670, 691, 694]), ([0, 945, 910], [670, 691, 698]), ([0, 945, 910], [670, 697, 685]), ([0, 945, 910], [670, 697, 698]), ([0, 945, 910], [670, 711, 663]), ([0, 945, 910], [670, 715, 667]), ([0, 945, 910], [670, 413, 663]), ([0, 945, 910], [670, 413, 667]), ([0, 945, 910], [670, 413, 669]), ([0, 945, 910], [670, 438, 694]), ([0, 945, 910], [670, 438, 698]), ([0, 945, 910], [670, 663, 669]), ([0, 945, 910], [670, 663, 679]), ([0, 945, 910], [670, 663, 694]), ([0, 945, 910], [670, 663, 700]), ([0, 945, 910], [670, 667, 698]), ([0, 945, 910], [670, 669, 694]), ([0, 945, 910], [670, 669, 700]), ([0, 945, 910], [670, 698, 700]), ([0, 945, 910], [686, 691, 693]), ([0, 945, 910], [686, 691, 663]), ([0, 945, 910], [686, 691, 679]), ([0, 945, 910], [686, 691, 683]), ([0, 945, 910], [686, 691, 700]), ([0, 945, 910], [686, 693, 697]), ([0, 945, 910], [686, 693, 679]), ([0, 945, 910], [686, 693, 685]), ([0, 945, 910], [686, 697, 683]), ([0, 945, 910], [686, 697, 685]), ([0, 945, 910], [686, 746, 683]), ([0, 945, 910], [686, 438, 700]), ([0, 945, 910], [686, 442, 698]), ([0, 945, 910], [686, 442, 700]), ([0, 945, 910], [686, 663, 679]), ([0, 945, 910], [686, 663, 700]), ([0, 945, 910], [686, 679, 683]), ([0, 945, 910], [686, 679, 685]), ([0, 945, 910], [686, 679, 698]), ([0, 945, 910], [686, 683, 685]), ([0, 945, 910], [686, 685, 698]), ([0, 945, 910], [686, 698, 700]), ([0, 945, 910], [691, 693, 697]), ([0, 945, 910], [691, 693, 694]), ([0, 945, 910], [691, 697, 663]), ([0, 945, 910], [691, 697, 667]), ([0, 945, 910], [691, 697, 685]), ([0, 945, 910], [691, 715, 683]), ([0, 945, 910], [691, 730, 667]), ([0, 945, 910], [691, 442, 683]), ([0, 945, 910], [691, 663, 669]), ([0, 945, 910], [691, 663, 685]), ([0, 945, 910], [691, 663, 694]), ([0, 945, 910], [691, 667, 683]), ([0, 945, 910], [691, 694, 698]), ([0, 945, 910], [691, 698, 700]), ([0, 945, 910], [693, 697, 669]), ([0, 945, 910], [693, 697, 700]), ([0, 945, 910], [693, 711, 663]), ([0, 945, 910], [693, 725, 679]), ([0, 945, 910], [693, 725, 694]), ([0, 945, 910], [693, 741, 663]), ([0, 945, 910], [693, 741, 700]), ([0, 945, 910], [693, 745, 685]), ([0, 945, 910], [693, 413, 663]), ([0, 945, 910], [693, 423, 694]), ([0, 945, 910], [693, 429, 679]), ([0, 945, 910], [693, 438, 694]), ([0, 945, 910], [693, 663, 679]), ([0, 945, 910], [693, 663, 694]), ([0, 945, 910], [693, 663, 700]), ([0, 945, 910], [693, 685, 700]), ([0, 945, 910], [693, 694, 700]), ([0, 945, 910], [697, 427, 667]), ([0, 945, 910], [697, 427, 698]), ([0, 945, 910], [697, 444, 700]), ([0, 945, 910], [697, 663, 700]), ([0, 945, 910], [697, 669, 685]), ([0, 945, 910], [697, 669, 698]), ([0, 945, 910], [697, 683, 698]), ([0, 945, 910], [697, 694, 700]), ([0, 945, 910], [723, 726, 663]), ([0, 945, 910], [411, 667, 683]), ([0, 945, 910], [438, 442, 700]), ([0, 945, 910], [438, 694, 698]), ([0, 945, 910], [444, 669, 700]), ([0, 945, 910], [444, 685, 700]), ([0, 945, 910], [667, 683, 685]), ([0, 945, 910], [667, 694, 698]), ([0, 945, 910], [159, 543]), ([0, 945, 910], [159, 655]), ([0, 945, 910], [159, 670]), ([0, 945, 910], [159, 686]), ([0, 945, 910], [159, 691]), ([0, 945, 910], [159, 693]), ([0, 945, 910], [159, 697]), ([0, 945, 910], [159, 411]), ([0, 945, 910], [159, 663]), ([0, 945, 910], [159, 667]), ([0, 945, 910], [159, 669]), ([0, 945, 910], [159, 683]), ([0, 945, 910], [175, 430]), ([0, 945, 910], [175, 717]), ([0, 945, 910], [175, 423]), ([0, 945, 910], [175, 679]), ([0, 945, 910], [175, 683]), ([0, 945, 910], [175, 685]), ([0, 945, 910], [183, 414]), ([0, 945, 910], [183, 435]), ([0, 945, 910], [183, 437]), ([0, 945, 910], [183, 691]), ([0, 945, 910], [183, 693]), ([0, 945, 910], [183, 407]), ([0, 945, 910], [183, 423]), ([0, 945, 910], [183, 663]), ([0, 945, 910], [183, 698]), ([0, 945, 910], [187, 655]), ([0, 945, 910], [187, 691]), ([0, 945, 910], [187, 411]), ([0, 945, 910], [187, 427]), ([0, 945, 910], [187, 438]), ([0, 945, 910], [187, 663]), ([0, 945, 910], [187, 667]), ([0, 945, 910], [187, 683]), ([0, 945, 910], [187, 698]), ([0, 945, 910], [189, 317]), ([0, 945, 910], [189, 437]), ([0, 945, 910], [189, 441]), ([0, 945, 910], [189, 686]), ([0, 945, 910], [189, 693]), ([0, 945, 910], [189, 413]), ([0, 945, 910], [189, 444]), ([0, 945, 910], [189, 685]), ([0, 945, 910], [190, 318]), ([0, 945, 910], [190, 430]), ([0, 945, 910], [190, 686]), ([0, 945, 910], [190, 691]), ([0, 945, 910], [190, 693]), ([0, 945, 910], [190, 438]), ([0, 945, 910], [190, 442]), ([0, 945, 910], [190, 444]), ([0, 945, 910], [190, 700]), ([0, 945, 910], [287, 399]), ([0, 945, 910], [287, 430]), ([0, 945, 910], [287, 441]), ([0, 945, 910], [287, 670]), ([0, 945, 910], [287, 407]), ([0, 945, 910], [287, 442]), ([0, 945, 910], [287, 663]), ([0, 945, 910], [287, 669]), ([0, 945, 910], [303, 399]), ([0, 945, 910], [303, 427]), ([0, 945, 910], [303, 429]), ([0, 945, 910], [303, 683]), ([0, 945, 910], [311, 437]), ([0, 945, 910], [311, 423]), ([0, 945, 910], [311, 694]), ([0, 945, 910], [315, 414]), ([0, 945, 910], [315, 435]), ([0, 945, 910], [315, 441]), ([0, 945, 910], [315, 691]), ([0, 945, 910], [315, 411]), ([0, 945, 910], [315, 427]), ([0, 945, 910], [315, 442]), ([0, 945, 910], [315, 683]), ([0, 945, 910], [315, 698]), ([0, 945, 910], [317, 437]), ([0, 945, 910], [317, 413]), ([0, 945, 910], [317, 438]), ([0, 945, 910], [317, 444]), ([0, 945, 910], [317, 685]), ([0, 945, 910], [318, 414]), ([0, 945, 910], [318, 430]), ([0, 945, 910], [318, 435]), ([0, 945, 910], [318, 574]), ([0, 945, 910], [318, 413]), ([0, 945, 910], [318, 423]), ([0, 945, 910], [318, 427]), ([0, 945, 910], [318, 438]), ([0, 945, 910], [318, 442]), ([0, 945, 910], [318, 444]), ([0, 945, 910], [318, 694]), ([0, 945, 910], [318, 698]), ([0, 945, 910], [318, 700]), ([0, 945, 910], [399, 414]), ([0, 945, 910], [399, 430]), ([0, 945, 910], [399, 435]), ([0, 945, 910], [399, 437]), ([0, 945, 910], [399, 441]), ([0, 945, 910], [399, 459]), ([0, 945, 910], [399, 461]), ([0, 945, 910], [399, 485]), ([0, 945, 910], [399, 670]), ([0, 945, 910], [399, 686]), ([0, 945, 910], [399, 717]), ([0, 945, 910], [399, 407]), ([0, 945, 910], [399, 411]), ([0, 945, 910], [399, 427]), ([0, 945, 910], [399, 429]), ([0, 945, 910], [399, 438]), ([0, 945, 910], [399, 442]), ([0, 945, 910], [399, 679]), ([0, 945, 910], [399, 683]), ([0, 945, 910], [414, 430]), ([0, 945, 910], [414, 435]), ([0, 945, 910], [414, 437]), ([0, 945, 910], [414, 441]), ([0, 945, 910], [414, 455]), ([0, 945, 910], [414, 467]), ([0, 945, 910], [414, 469]), ([0, 945, 910], [414, 470]), ([0, 945, 910], [414, 474]), ([0, 945, 910], [414, 476]), ([0, 945, 910], [414, 486]), ([0, 945, 910], [414, 490]), ([0, 945, 910], [414, 498]), ([0, 945, 910], [414, 500]), ([0, 945, 910], [414, 504]), ([0, 945, 910], [414, 543]), ([0, 945, 910], [414, 655]), ([0, 945, 910], [414, 670]), ([0, 945, 910], [414, 686]), ([0, 945, 910], [414, 730]), ([0, 945, 910], [414, 732]), ([0, 945, 910], [414, 407]), ([0, 945, 910], [414, 411]), ([0, 945, 910], [414, 413]), ([0, 945, 910], [414, 423]), ([0, 945, 910], [414, 429]), ([0, 945, 910], [414, 438]), ([0, 945, 910], [414, 442]), ([0, 945, 910], [414, 444]), ([0, 945, 910], [414, 663]), ([0, 945, 910], [414, 667]), ([0, 945, 910], [414, 669]), ([0, 945, 910], [414, 694]), ([0, 945, 910], [414, 698]), ([0, 945, 910], [414, 700]), ([0, 945, 910], [430, 467]), ([0, 945, 910], [430, 435]), ([0, 945, 910], [430, 459]), ([0, 945, 910], [430, 461]), ([0, 945, 910], [430, 470]), ([0, 945, 910], [430, 476]), ([0, 945, 910], [430, 483]), ([0, 945, 910], [430, 485]), ([0, 945, 910], [430, 486]), ([0, 945, 910], [430, 489]), ([0, 945, 910], [430, 490]), ([0, 945, 910], [430, 492]), ([0, 945, 910], [430, 498]), ([0, 945, 910], [430, 500]), ([0, 945, 910], [430, 655]), ([0, 945, 910], [430, 670]), ([0, 945, 910], [430, 686]), ([0, 945, 910], [430, 742]), ([0, 945, 910], [430, 411]), ([0, 945, 910], [430, 413]), ([0, 945, 910], [430, 423]), ([0, 945, 910], [430, 427]), ([0, 945, 910], [430, 429]), ([0, 945, 910], [430, 438]), ([0, 945, 910], [430, 442]), ([0, 945, 910], [430, 444]), ([0, 945, 910], [430, 679]), ([0, 945, 910], [430, 683]), ([0, 945, 910], [430, 685]), ([0, 945, 910], [430, 694]), ([0, 945, 910], [430, 700]), ([0, 945, 910], [435, 437]), ([0, 945, 910], [435, 467]), ([0, 945, 910], [435, 470]), ([0, 945, 910], [435, 474]), ([0, 945, 910], [435, 483]), ([0, 945, 910], [435, 500]), ([0, 945, 910], [435, 693]), ([0, 945, 910], [435, 697]), ([0, 945, 910], [435, 723]), ([0, 945, 910], [435, 739]), ([0, 945, 910], [435, 407]), ([0, 945, 910], [435, 411]), ([0, 945, 910], [435, 413]), ([0, 945, 910], [435, 423]), ([0, 945, 910], [435, 427]), ([0, 945, 910], [435, 438]), ([0, 945, 910], [435, 442]), ([0, 945, 910], [435, 444]), ([0, 945, 910], [435, 667]), ([0, 945, 910], [435, 679]), ([0, 945, 910], [435, 683]), ([0, 945, 910], [435, 694]), ([0, 945, 910], [435, 698]), ([0, 945, 910], [437, 686]), ([0, 945, 910], [437, 441]), ([0, 945, 910], [437, 455]), ([0, 945, 910], [437, 461]), ([0, 945, 910], [437, 476]), ([0, 945, 910], [437, 485]), ([0, 945, 910], [437, 486]), ([0, 945, 910], [437, 500]), ([0, 945, 910], [437, 691]), ([0, 945, 910], [437, 693]), ([0, 945, 910], [437, 697]), ([0, 945, 910], [437, 741]), ([0, 945, 910], [437, 407]), ([0, 945, 910], [437, 411]), ([0, 945, 910], [437, 413]), ([0, 945, 910], [437, 423]), ([0, 945, 910], [437, 427]), ([0, 945, 910], [437, 429]), ([0, 945, 910], [437, 438]), ([0, 945, 910], [437, 442]), ([0, 945, 910], [437, 444]), ([0, 945, 910], [437, 669]), ([0, 945, 910], [437, 679]), ([0, 945, 910], [437, 685]), ([0, 945, 910], [437, 694]), ([0, 945, 910], [437, 700]), ([0, 945, 910], [441, 459]), ([0, 945, 910], [441, 461]), ([0, 945, 910], [441, 473]), ([0, 945, 910], [441, 476]), ([0, 945, 910], [441, 489]), ([0, 945, 910], [441, 490]), ([0, 945, 910], [441, 504]), ([0, 945, 910], [441, 691]), ([0, 945, 910], [441, 694]), ([0, 945, 910], [441, 697]), ([0, 945, 910], [441, 745]), ([0, 945, 910], [441, 411]), ([0, 945, 910], [441, 413]), ([0, 945, 910], [441, 427]), ([0, 945, 910], [441, 429]), ([0, 945, 910], [441, 438]), ([0, 945, 910], [441, 442]), ([0, 945, 910], [441, 444]), ([0, 945, 910], [441, 667]), ([0, 945, 910], [441, 669]), ([0, 945, 910], [441, 698]), ([0, 945, 910], [441, 700]), ([0, 945, 910], [455, 461]), ([0, 945, 910], [455, 469]), ([0, 945, 910], [455, 483]), ([0, 945, 910], [455, 485]), ([0, 945, 910], [455, 655]), ([0, 945, 910], [455, 723]), ([0, 945, 910], [455, 407]), ([0, 945, 910], [455, 423]), ([0, 945, 910], [455, 438]), ([0, 945, 910], [459, 473]), ([0, 945, 910], [459, 483]), ([0, 945, 910], [459, 655]), ([0, 945, 910], [459, 715]), ([0, 945, 910], [459, 729]), ([0, 945, 910], [459, 739]), ([0, 945, 910], [459, 407]), ([0, 945, 910], [459, 411]), ([0, 945, 910], [459, 413]), ([0, 945, 910], [459, 442]), ([0, 945, 910], [459, 683]), ([0, 945, 910], [461, 470]), ([0, 945, 910], [461, 473]), ([0, 945, 910], [461, 476]), ([0, 945, 910], [461, 485]), ([0, 945, 910], [461, 717]), ([0, 945, 910], [461, 413]), ([0, 945, 910], [461, 429]), ([0, 945, 910], [467, 670]), ([0, 945, 910], [467, 474]), ([0, 945, 910], [467, 483]), ([0, 945, 910], [467, 711]), ([0, 945, 910], [467, 407]), ([0, 945, 910], [467, 411]), ([0, 945, 910], [467, 667]), ([0, 945, 910], [469, 476]), ([0, 945, 910], [469, 693]), ([0, 945, 910], [469, 725]), ([0, 945, 910], [469, 407]), ([0, 945, 910], [469, 411]), ([0, 945, 910], [469, 413]), ([0, 945, 910], [469, 423]), ([0, 945, 910], [469, 438]), ([0, 945, 910], [470, 474]), ([0, 945, 910], [470, 476]), ([0, 945, 910], [470, 498]), ([0, 945, 910], [470, 500]), ([0, 945, 910], [470, 670]), ([0, 945, 910], [470, 730]), ([0, 945, 910], [470, 423]), ([0, 945, 910], [470, 438]), ([0, 945, 910], [470, 444]), ([0, 945, 910], [470, 694]), ([0, 945, 910], [473, 474]), ([0, 945, 910], [473, 476]), ([0, 945, 910], [473, 485]), ([0, 945, 910], [473, 504]), ([0, 945, 910], [473, 411]), ([0, 945, 910], [473, 413]), ([0, 945, 910], [473, 667]), ([0, 945, 910], [473, 669]), ([0, 945, 910], [474, 476]), ([0, 945, 910], [474, 498]), ([0, 945, 910], [474, 504]), ([0, 945, 910], [474, 411]), ([0, 945, 910], [474, 413]), ([0, 945, 910], [474, 427]), ([0, 945, 910], [474, 438]), ([0, 945, 910], [474, 442]), ([0, 945, 910], [474, 444]), ([0, 945, 910], [474, 698]), ([0, 945, 910], [476, 500]), ([0, 945, 910], [476, 504]), ([0, 945, 910], [476, 717]), ([0, 945, 910], [476, 730]), ([0, 945, 910], [476, 413]), ([0, 945, 910], [476, 429]), ([0, 945, 910], [476, 438]), ([0, 945, 910], [476, 442]), ([0, 945, 910], [476, 444]), ([0, 945, 910], [476, 700]), ([0, 945, 910], [483, 485]), ([0, 945, 910], [483, 486]), ([0, 945, 910], [483, 490]), ([0, 945, 910], [483, 498]), ([0, 945, 910], [483, 745]), ([0, 945, 910], [483, 407]), ([0, 945, 910], [483, 423]), ([0, 945, 910], [483, 429]), ([0, 945, 910], [485, 486]), ([0, 945, 910], [485, 492]), ([0, 945, 910], [485, 413]), ([0, 945, 910], [485, 423]), ([0, 945, 910], [485, 429]), ([0, 945, 910], [485, 438]), ([0, 945, 910], [485, 444]), ([0, 945, 910], [486, 492]), ([0, 945, 910], [486, 500]), ([0, 945, 910], [486, 423]), ([0, 945, 910], [486, 427]), ([0, 945, 910], [486, 438]), ([0, 945, 910], [486, 442]), ([0, 945, 910], [486, 444]), ([0, 945, 910], [486, 679]), ([0, 945, 910], [486, 694]), ([0, 945, 910], [489, 490]), ([0, 945, 910], [489, 492]), ([0, 945, 910], [489, 498]), ([0, 945, 910], [489, 500]), ([0, 945, 910], [489, 729]), ([0, 945, 910], [489, 429]), ([0, 945, 910], [489, 442]), ([0, 945, 910], [489, 444]), ([0, 945, 910], [489, 683]), ([0, 945, 910], [490, 498]), ([0, 945, 910], [490, 504]), ([0, 945, 910], [490, 715]), ([0, 945, 910], [490, 745]), ([0, 945, 910], [490, 411]), ([0, 945, 910], [490, 423]), ([0, 945, 910], [490, 427]), ([0, 945, 910], [490, 438]), ([0, 945, 910], [490, 442]), ([0, 945, 910], [490, 444]), ([0, 945, 910], [490, 683]), ([0, 945, 910], [490, 698]), ([0, 945, 910], [492, 500]), ([0, 945, 910], [492, 504]), ([0, 945, 910], [492, 413]), ([0, 945, 910], [492, 427]), ([0, 945, 910], [492, 438]), ([0, 945, 910], [492, 442]), ([0, 945, 910], [492, 444]), ([0, 945, 910], [492, 685]), ([0, 945, 910], [498, 500]), ([0, 945, 910], [498, 691]), ([0, 945, 910], [498, 411]), ([0, 945, 910], [498, 423]), ([0, 945, 910], [498, 438]), ([0, 945, 910], [498, 442]), ([0, 945, 910], [498, 444]), ([0, 945, 910], [498, 694]), ([0, 945, 910], [500, 693]), ([0, 945, 910], [500, 741]), ([0, 945, 910], [500, 413]), ([0, 945, 910], [500, 423]), ([0, 945, 910], [500, 429]), ([0, 945, 910], [500, 438]), ([0, 945, 910], [500, 442]), ([0, 945, 910], [500, 444]), ([0, 945, 910], [500, 694]), ([0, 945, 910], [500, 700]), ([0, 945, 910], [504, 670]), ([0, 945, 910], [504, 697]), ([0, 945, 910], [504, 438]), ([0, 945, 910], [504, 442]), ([0, 945, 910], [504, 444]), ([0, 945, 910], [543, 567]), ([0, 945, 910], [543, 571]), ([0, 945, 910], [543, 573]), ([0, 945, 910], [543, 574]), ([0, 945, 910], [543, 655]), ([0, 945, 910], [543, 670]), ([0, 945, 910], [543, 691]), ([0, 945, 910], [543, 730]), ([0, 945, 910], [543, 407]), ([0, 945, 910], [543, 411]), ([0, 945, 910], [543, 663]), ([0, 945, 910], [543, 667]), ([0, 945, 910], [543, 669]), ([0, 945, 910], [543, 683]), ([0, 945, 910], [559, 655]), ([0, 945, 910], [559, 670]), ([0, 945, 910], [559, 686]), ([0, 945, 910], [559, 423]), ([0, 945, 910], [559, 427]), ([0, 945, 910], [559, 663]), ([0, 945, 910], [559, 679]), ([0, 945, 910], [559, 683]), ([0, 945, 910], [559, 685]), ([0, 945, 910], [559, 700]), ([0, 945, 910], [567, 574]), ([0, 945, 910], [567, 686]), ([0, 945, 910], [567, 691]), ([0, 945, 910], [567, 693]), ([0, 945, 910], [567, 407]), ([0, 945, 910], [567, 423]), ([0, 945, 910], [567, 663]), ([0, 945, 910], [567, 669]), ([0, 945, 910], [567, 679]), ([0, 945, 910], [567, 698]), ([0, 945, 910], [571, 573]), ([0, 945, 910], [571, 574]), ([0, 945, 910], [571, 655]), ([0, 945, 910], [571, 670]), ([0, 945, 910], [571, 697]), ([0, 945, 910], [571, 427]), ([0, 945, 910], [571, 442]), ([0, 945, 910], [571, 663]), ([0, 945, 910], [571, 667]), ([0, 945, 910], [571, 683]), ([0, 945, 910], [571, 685]), ([0, 945, 910], [571, 698]), ([0, 945, 910], [571, 700]), ([0, 945, 910], [573, 574]), ([0, 945, 910], [573, 697]), ([0, 945, 910], [573, 413]), ([0, 945, 910], [573, 429]), ([0, 945, 910], [573, 669]), ([0, 945, 910], [573, 679]), ([0, 945, 910], [573, 683]), ([0, 945, 910], [573, 685]), ([0, 945, 910], [574, 670]), ([0, 945, 910], [574, 686]), ([0, 945, 910], [574, 691]), ([0, 945, 910], [574, 438]), ([0, 945, 910], [574, 444]), ([0, 945, 910], [574, 663]), ([0, 945, 910], [574, 667]), ([0, 945, 910], [574, 669]), ([0, 945, 910], [574, 679]), ([0, 945, 910], [574, 683]), ([0, 945, 910], [574, 694]), ([0, 945, 910], [574, 698]), ([0, 945, 910], [574, 700]), ([0, 945, 910], [655, 670]), ([0, 945, 910], [655, 686]), ([0, 945, 910], [655, 691]), ([0, 945, 910], [655, 693]), ([0, 945, 910], [655, 697]), ([0, 945, 910], [655, 711]), ([0, 945, 910], [655, 715]), ([0, 945, 910], [655, 717]), ([0, 945, 910], [655, 729]), ([0, 945, 910], [655, 730]), ([0, 945, 910], [655, 739]), ([0, 945, 910], [655, 741]), ([0, 945, 910], [655, 745]), ([0, 945, 910], [655, 746]), ([0, 945, 910], [655, 407]), ([0, 945, 910], [655, 411]), ([0, 945, 910], [655, 413]), ([0, 945, 910], [655, 423]), ([0, 945, 910], [655, 429]), ([0, 945, 910], [655, 663]), ([0, 945, 910], [655, 667]), ([0, 945, 910], [655, 669]), ([0, 945, 910], [655, 679]), ([0, 945, 910], [655, 683]), ([0, 945, 910], [655, 685]), ([0, 945, 910], [655, 698]), ([0, 945, 910], [670, 686]), ([0, 945, 910], [670, 691]), ([0, 945, 910], [670, 693]), ([0, 945, 910], [670, 697]), ([0, 945, 910], [670, 711]), ([0, 945, 910], [670, 715]), ([0, 945, 910], [670, 717]), ([0, 945, 910], [670, 725]), ([0, 945, 910], [670, 726]), ([0, 945, 910], [670, 729]), ([0, 945, 910], [670, 730]), ([0, 945, 910], [670, 732]), ([0, 945, 910], [670, 742]), ([0, 945, 910], [670, 407]), ([0, 945, 910], [670, 411]), ([0, 945, 910], [670, 413]), ([0, 945, 910], [670, 438]), ([0, 945, 910], [670, 442]), ([0, 945, 910], [670, 444]), ([0, 945, 910], [670, 663]), ([0, 945, 910], [670, 667]), ([0, 945, 910], [670, 669]), ([0, 945, 910], [670, 679]), ([0, 945, 910], [670, 683]), ([0, 945, 910], [670, 685]), ([0, 945, 910], [670, 694]), ([0, 945, 910], [670, 698]), ([0, 945, 910], [670, 700]), ([0, 945, 910], [686, 691]), ([0, 945, 910], [686, 693]), ([0, 945, 910], [686, 697]), ([0, 945, 910], [686, 715]), ([0, 945, 910], [686, 717]), ([0, 945, 910], [686, 726]), ([0, 945, 910], [686, 730]), ([0, 945, 910], [686, 739]), ([0, 945, 910], [686, 742]), ([0, 945, 910], [686, 745]), ([0, 945, 910], [686, 746]), ([0, 945, 910], [686, 423]), ([0, 945, 910], [686, 427]), ([0, 945, 910], [686, 438]), ([0, 945, 910], [686, 442]), ([0, 945, 910], [686, 444]), ([0, 945, 910], [686, 663]), ([0, 945, 910], [686, 667]), ([0, 945, 910], [686, 679]), ([0, 945, 910], [686, 683]), ([0, 945, 910], [686, 685]), ([0, 945, 910], [686, 694]), ([0, 945, 910], [686, 698]), ([0, 945, 910], [686, 700]), ([0, 945, 910], [691, 717]), ([0, 945, 910], [691, 693]), ([0, 945, 910], [691, 711]), ([0, 945, 910], [691, 715]), ([0, 945, 910], [691, 723]), ([0, 945, 910], [691, 730]), ([0, 945, 910], [691, 741]), ([0, 945, 910], [691, 746]), ([0, 945, 910], [691, 407]), ([0, 945, 910], [691, 423]), ([0, 945, 910], [691, 427]), ([0, 945, 910], [691, 442]), ([0, 945, 910], [691, 663]), ([0, 945, 910], [691, 667]), ([0, 945, 910], [691, 669]), ([0, 945, 910], [691, 679]), ([0, 945, 910], [691, 683]), ([0, 945, 910], [691, 685]), ([0, 945, 910], [691, 694]), ([0, 945, 910], [691, 698]), ([0, 945, 910], [691, 700]), ([0, 945, 910], [693, 746]), ([0, 945, 910], [693, 697]), ([0, 945, 910], [693, 717]), ([0, 945, 910], [693, 725]), ([0, 945, 910], [693, 726]), ([0, 945, 910], [693, 732]), ([0, 945, 910], [693, 745]), ([0, 945, 910], [693, 407]), ([0, 945, 910], [693, 413]), ([0, 945, 910], [693, 423]), ([0, 945, 910], [693, 429]), ([0, 945, 910], [693, 444]), ([0, 945, 910], [693, 663]), ([0, 945, 910], [693, 667]), ([0, 945, 910], [693, 669]), ([0, 945, 910], [693, 679]), ([0, 945, 910], [693, 683]), ([0, 945, 910], [693, 694]), ([0, 945, 910], [693, 698]), ([0, 945, 910], [693, 700]), ([0, 945, 910], [697, 423]), ([0, 945, 910], [697, 715]), ([0, 945, 910], [697, 723]), ([0, 945, 910], [697, 730]), ([0, 945, 910], [697, 732]), ([0, 945, 910], [697, 739]), ([0, 945, 910], [697, 741]), ([0, 945, 910], [697, 745]), ([0, 945, 910], [697, 411]), ([0, 945, 910], [697, 413]), ([0, 945, 910], [697, 427]), ([0, 945, 910], [697, 442]), ([0, 945, 910], [697, 444]), ([0, 945, 910], [697, 663]), ([0, 945, 910], [697, 667]), ([0, 945, 910], [697, 669]), ([0, 945, 910], [697, 679]), ([0, 945, 910], [697, 683]), ([0, 945, 910], [697, 685]), ([0, 945, 910], [697, 694]), ([0, 945, 910], [697, 698]), ([0, 945, 910], [697, 700]), ([0, 945, 910], [711, 715]), ([0, 945, 910], [711, 717]), ([0, 945, 910], [711, 725]), ([0, 945, 910], [711, 739]), ([0, 945, 910], [711, 741]), ([0, 945, 910], [711, 742]), ([0, 945, 910], [711, 745]), ([0, 945, 910], [711, 746]), ([0, 945, 910], [711, 423]), ([0, 945, 910], [711, 663]), ([0, 945, 910], [711, 667]), ([0, 945, 910], [711, 669]), ([0, 945, 910], [711, 685]), ([0, 945, 910], [711, 694]), ([0, 945, 910], [715, 717]), ([0, 945, 910], [715, 723]), ([0, 945, 910], [715, 725]), ([0, 945, 910], [715, 729]), ([0, 945, 910], [715, 730]), ([0, 945, 910], [715, 745]), ([0, 945, 910], [715, 411]), ([0, 945, 910], [715, 427]), ([0, 945, 910], [715, 663]), ([0, 945, 910], [715, 667]), ([0, 945, 910], [715, 683]), ([0, 945, 910], [715, 685]), ([0, 945, 910], [717, 725]), ([0, 945, 910], [717, 732]), ([0, 945, 910], [717, 741]), ([0, 945, 910], [717, 745]), ([0, 945, 910], [717, 413]), ([0, 945, 910], [717, 429]), ([0, 945, 910], [717, 667]), ([0, 945, 910], [717, 669]), ([0, 945, 910], [717, 679]), ([0, 945, 910], [717, 700]), ([0, 945, 910], [723, 729]), ([0, 945, 910], [723, 739]), ([0, 945, 910], [723, 407]), ([0, 945, 910], [723, 411]), ([0, 945, 910], [723, 663]), ([0, 945, 910], [723, 669]), ([0, 945, 910], [723, 683]), ([0, 945, 910], [725, 726]), ([0, 945, 910], [725, 729]), ([0, 945, 910], [725, 732]), ([0, 945, 910], [725, 741]), ([0, 945, 910], [725, 742]), ([0, 945, 910], [725, 413]), ([0, 945, 910], [725, 663]), ([0, 945, 910], [725, 667]), ([0, 945, 910], [725, 669]), ([0, 945, 910], [725, 685]), ([0, 945, 910], [725, 694]), ([0, 945, 910], [726, 732]), ([0, 945, 910], [726, 741]), ([0, 945, 910], [726, 663]), ([0, 945, 910], [726, 667]), ([0, 945, 910], [726, 679]), ([0, 945, 910], [726, 694]), ([0, 945, 910], [726, 698]), ([0, 945, 910], [729, 730]), ([0, 945, 910], [729, 411]), ([0, 945, 910], [729, 413]), ([0, 945, 910], [729, 663]), ([0, 945, 910], [729, 667]), ([0, 945, 910], [729, 669]), ([0, 945, 910], [729, 683]), ([0, 945, 910], [729, 698]), ([0, 945, 910], [729, 700]), ([0, 945, 910], [730, 732]), ([0, 945, 910], [730, 745]), ([0, 945, 910], [730, 442]), ([0, 945, 910], [730, 663]), ([0, 945, 910], [730, 667]), ([0, 945, 910], [730, 669]), ([0, 945, 910], [730, 683]), ([0, 945, 910], [730, 694]), ([0, 945, 910], [730, 700]), ([0, 945, 910], [732, 413]), ([0, 945, 910], [732, 663]), ([0, 945, 910], [732, 669]), ([0, 945, 910], [732, 685]), ([0, 945, 910], [732, 698]), ([0, 945, 910], [739, 669]), ([0, 945, 910], [739, 700]), ([0, 945, 910], [739, 746]), ([0, 945, 910], [739, 423]), ([0, 945, 910], [739, 663]), ([0, 945, 910], [739, 667]), ([0, 945, 910], [739, 683]), ([0, 945, 910], [739, 685]), ([0, 945, 910], [739, 694]), ([0, 945, 910], [741, 423]), ([0, 945, 910], [741, 663]), ([0, 945, 910], [741, 679]), ([0, 945, 910], [741, 683]), ([0, 945, 910], [741, 685]), ([0, 945, 910], [741, 694]), ([0, 945, 910], [741, 700]), ([0, 945, 910], [742, 423]), ([0, 945, 910], [742, 663]), ([0, 945, 910], [742, 679]), ([0, 945, 910], [742, 683]), ([0, 945, 910], [742, 685]), ([0, 945, 910], [742, 694]), ([0, 945, 910], [742, 700]), ([0, 945, 910], [745, 429]), ([0, 945, 910], [745, 669]), ([0, 945, 910], [745, 683]), ([0, 945, 910], [745, 698]), ([0, 945, 910], [745, 700]), ([0, 945, 910], [746, 442]), ([0, 945, 910], [746, 667]), ([0, 945, 910], [746, 683]), ([0, 945, 910], [746, 685]), ([0, 945, 910], [746, 700]), ([0, 945, 910], [407, 413]), ([0, 945, 910], [407, 423]), ([0, 945, 910], [407, 427]), ([0, 945, 910], [407, 429]), ([0, 945, 910], [407, 438]), ([0, 945, 910], [407, 444]), ([0, 945, 910], [407, 663]), ([0, 945, 910], [407, 667]), ([0, 945, 910], [407, 669]), ([0, 945, 910], [407, 679]), ([0, 945, 910], [407, 694]), ([0, 945, 910], [411, 413]), ([0, 945, 910], [411, 423]), ([0, 945, 910], [411, 427]), ([0, 945, 910], [411, 429]), ([0, 945, 910], [411, 438]), ([0, 945, 910], [411, 442]), ([0, 945, 910], [411, 444]), ([0, 945, 910], [411, 663]), ([0, 945, 910], [411, 667]), ([0, 945, 910], [411, 669]), ([0, 945, 910], [411, 683]), ([0, 945, 910], [413, 683]), ([0, 945, 910], [413, 423]), ([0, 945, 910], [413, 427]), ([0, 945, 910], [413, 429]), ([0, 945, 910], [413, 438]), ([0, 945, 910], [413, 442]), ([0, 945, 910], [413, 444]), ([0, 945, 910], [413, 663]), ([0, 945, 910], [413, 667]), ([0, 945, 910], [413, 700]), ([0, 945, 910], [423, 427]), ([0, 945, 910], [423, 429]), ([0, 945, 910], [423, 438]), ([0, 945, 910], [423, 442]), ([0, 945, 910], [423, 444]), ([0, 945, 910], [423, 679]), ([0, 945, 910], [423, 683]), ([0, 945, 910], [423, 685]), ([0, 945, 910], [423, 694]), ([0, 945, 910], [427, 429]), ([0, 945, 910], [427, 438]), ([0, 945, 910], [427, 442]), ([0, 945, 910], [427, 444]), ([0, 945, 910], [427, 667]), ([0, 945, 910], [427, 679]), ([0, 945, 910], [427, 683]), ([0, 945, 910], [427, 685]), ([0, 945, 910], [427, 698]), ([0, 945, 910], [429, 442]), ([0, 945, 910], [429, 444]), ([0, 945, 910], [429, 679]), ([0, 945, 910], [429, 683]), ([0, 945, 910], [429, 685]), ([0, 945, 910], [438, 442]), ([0, 945, 910], [438, 444]), ([0, 945, 910], [438, 663]), ([0, 945, 910], [438, 679]), ([0, 945, 910], [438, 694]), ([0, 945, 910], [438, 698]), ([0, 945, 910], [438, 700]), ([0, 945, 910], [442, 444]), ([0, 945, 910], [442, 667]), ([0, 945, 910], [442, 683]), ([0, 945, 910], [442, 698]), ([0, 945, 910], [442, 700]), ([0, 945, 910], [444, 663]), ([0, 945, 910], [444, 669]), ([0, 945, 910], [444, 685]), ([0, 945, 910], [444, 694]), ([0, 945, 910], [444, 698]), ([0, 945, 910], [444, 700]), ([0, 945, 910], [663, 667]), ([0, 945, 910], [663, 669]), ([0, 945, 910], [663, 679]), ([0, 945, 910], [663, 683]), ([0, 945, 910], [663, 685]), ([0, 945, 910], [663, 698]), ([0, 945, 910], [663, 700]), ([0, 945, 910], [667, 669]), ([0, 945, 910], [667, 679]), ([0, 945, 910], [667, 683]), ([0, 945, 910], [667, 685]), ([0, 945, 910], [667, 698]), ([0, 945, 910], [667, 700]), ([0, 945, 910], [669, 679]), ([0, 945, 910], [669, 683]), ([0, 945, 910], [669, 685]), ([0, 945, 910], [669, 694]), ([0, 945, 910], [669, 698]), ([0, 945, 910], [669, 700]), ([0, 945, 910], [679, 683]), ([0, 945, 910], [679, 685]), ([0, 945, 910], [679, 694]), ([0, 945, 910], [679, 700]), ([0, 945, 910], [683, 685]), ([0, 945, 910], [683, 698]), ([0, 945, 910], [683, 700]), ([0, 945, 910], [685, 694]), ([0, 945, 910], [685, 698]), ([0, 945, 910], [685, 700]), ([0, 945, 910], [694, 698]), ([0, 945, 910], [694, 700]), ([0, 945, 910], [698, 700]), ([0, 945, 910], [159]), ([0, 945, 910], [175]), ([0, 945, 910], [183]), ([0, 945, 910], [187]), ([0, 945, 910], [189]), ([0, 945, 910], [190]), ([0, 945, 910], [287]), ([0, 945, 910], [303]), ([0, 945, 910], [311]), ([0, 945, 910], [315]), ([0, 945, 910], [317]), ([0, 945, 910], [318]), ([0, 945, 910], [399]), ([0, 945, 910], [414]), ([0, 945, 910], [430]), ([0, 945, 910], [435]), ([0, 945, 910], [437]), ([0, 945, 910], [441]), ([0, 945, 910], [455]), ([0, 945, 910], [459]), ([0, 945, 910], [461]), ([0, 945, 910], [467]), ([0, 945, 910], [469]), ([0, 945, 910], [470]), ([0, 945, 910], [473]), ([0, 945, 910], [474]), ([0, 945, 910], [476]), ([0, 945, 910], [483]), ([0, 945, 910], [485]), ([0, 945, 910], [486]), ([0, 945, 910], [489]), ([0, 945, 910], [490]), ([0, 945, 910], [492]), ([0, 945, 910], [498]), ([0, 945, 910], [500]), ([0, 945, 910], [504]), ([0, 945, 910], [543]), ([0, 945, 910], [559]), ([0, 945, 910], [567]), ([0, 945, 910], [571]), ([0, 945, 910], [573]), ([0, 945, 910], [574]), ([0, 945, 910], [655]), ([0, 945, 910], [670]), ([0, 945, 910], [686]), ([0, 945, 910], [691]), ([0, 945, 910], [693]), ([0, 945, 910], [697]), ([0, 945, 910], [711]), ([0, 945, 910], [715]), ([0, 945, 910], [717]), ([0, 945, 910], [723]), ([0, 945, 910], [725]), ([0, 945, 910], [726]), ([0, 945, 910], [729]), ([0, 945, 910], [730]), ([0, 945, 910], [732]), ([0, 945, 910], [739]), ([0, 945, 910], [741]), ([0, 945, 910], [742]), ([0, 945, 910], [745]), ([0, 945, 910], [746]), ([0, 945, 910], [407]), ([0, 945, 910], [411]), ([0, 945, 910], [413]), ([0, 945, 910], [423]), ([0, 945, 910], [427]), ([0, 945, 910], [429]), ([0, 945, 910], [438]), ([0, 945, 910], [442]), ([0, 945, 910], [444]), ([0, 945, 910], [663]), ([0, 945, 910], [667]), ([0, 945, 910], [669]), ([0, 945, 910], [679]), ([0, 945, 910], [683]), ([0, 945, 910], [685]), ([0, 945, 910], [694]), ([0, 945, 910], [698]), ([0, 945, 910], [700])]))

1046


In [49]:
%%time

#решаем вопрос о ровно 3 вершинах для случая (1 из первых 80), (+2 из остальных). Этот случай вложен в (1 из первых 80) (+1+ из остальных)
SSAVED = []
for gr in SAVED:
    if len(gr[1]) == 1:
        k2_ss, k3_ss = build_blocks_by_conf(gr)
        BIG_NUM = 2 - len(gr[1])
        bruteforce_nolimits_nok4(to_check=k_6[80:], base_conf=gr[0], add_conf=gr[1], k_2=k_2, k_4=k_4, k2s=k2_ss, kl3s=k2_ss, how_much=BIG_NUM, timer=20, n=10, k=6)
 

[[0, 0], [57, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [59, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [61, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [57, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [68, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [46, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [48, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [61, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [57, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [53, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [68, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [64, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [60, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [53, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [49, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [63, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [68, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [46, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [53, 70], [54, 70], [0, 0], [0, 0], [

In [50]:
print(len(SSAVED), len(SAVED))

0 1046


In [51]:
#решаем вопрос о ровно 3 вершинах для случая (>>0<< из первых 80)
## вместо этого добираем +1+ из оставшихся на расстоянии 6
k2_ss, k3_ss = build_blocks_by_conf([[0, 945, 910],[]])
BIG_NUM = 1
bruteforce_nolimits_nok4(to_check=k_6[80:], base_conf=[0, 945, 910], add_conf=[], k_2=k_2, k_4=k_4, k2s=k2_ss, kl3s=k2_ss, how_much=BIG_NUM, timer=20, n=10, k=6)

In [52]:
print(len(SSAVED), len(SAVED))

0 1046


In [53]:
%%time

#решаем вопрос о хотя бы 5 вершинах для случаев (>>0,1 разобраны выше на самом деле<<,2,3 из первых 80)
## вместо этого добираем +1+ из оставшихся на расстоянии 6
for gr in SAVED:
    if len(gr[1]) == 2 or  len(gr[1]) == 3:
        k2_ss, k3_ss = build_blocks_by_conf(gr)
        BIG_NUM = 1
        bruteforce_nolimits_nok4(to_check=k_6[80:], base_conf=gr[0], add_conf=gr[1], k_2=k_2, k_4=k_4, k2s=k2_ss, kl3s=k2_ss, how_much=BIG_NUM, timer=20, n=10, k=6)
 

[[0, 0], [58, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [58, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [59, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [64, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [69, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [62, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [59, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [58, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [69, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [64, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [64, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [65, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [59, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [63, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [64, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [64, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [64, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [61, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [52, 70], [54, 70], [0, 0], [0, 0], [

[[0, 0], [59, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [55, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [66, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [63, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [62, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [56, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [66, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [59, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [67, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [68, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [59, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [59, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [69, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [65, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [61, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [57, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [61, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [60, 70], [54, 70], [0, 0], [0, 0], [0, 0]]
[[0, 0], [67, 70], [54, 70], [0, 0], [0, 0], [

In [54]:
print(len(SSAVED), len(SAVED))

0 1046
